# setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%%capture
!pip install git+https://github.com/EleutherAI/sae.git

In [3]:
# you should load this before cloning repo files
# from .config import SaeConfig
# from .utils import decoder_impl

from sae.config import SaeConfig
from sae.utils import decoder_impl
from sae import Sae

Triton not installed, using eager implementation of SAE decoder.


In [4]:
import pickle
from google.colab import files

import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

from collections import Counter
import pandas as pd
from IPython.display import display

import pdb

## corr fns

In [5]:
def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    if torch.cuda.is_available():
        reshaped_activations_A = reshaped_activations_A.to('cuda')
        reshaped_activations_B = reshaped_activations_B.to('cuda')

    # Normalize columns of A
    mean_A = reshaped_activations_A.mean(dim=0, keepdim=True)
    std_A = reshaped_activations_A.std(dim=0, keepdim=True)
    normalized_A = (reshaped_activations_A - mean_A) / (std_A + 1e-8)  # Avoid division by zero

    # Normalize columns of B
    mean_B = reshaped_activations_B.mean(dim=0, keepdim=True)
    std_B = reshaped_activations_B.std(dim=0, keepdim=True)
    normalized_B = (reshaped_activations_B - mean_B) / (std_B + 1e-8)  # Avoid division by zero

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        end = min(start + batch_size, normalized_B.shape[1])
        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        max_indices.append(max_idx)

        del batch_corr_matrix
        torch.cuda.empty_cache()

    corr_inds = torch.cat(max_indices).detach().cpu().numpy()
    corr_vals = torch.cat(max_values).detach().cpu().numpy()
    return corr_inds, corr_vals

In [6]:
def filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats):
    filt_corr_ind_A = []
    filt_corr_ind_B = []
    seen = set()
    for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
        if ind_A in kept_modA_feats:
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
        elif ind_A not in seen:  # only keep one if it's over count X
            seen.add(ind_A)
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
    num_unq_pairs = len(list(set(filt_corr_ind_A)))
    print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
    print("num 1-1 feats after filt: ", num_unq_pairs )
    return filt_corr_ind_A, filt_corr_ind_B, num_unq_pairs

In [7]:
def get_new_mean_corr(modA_feats, modB_feats, corr_vals):
    new_vals = []
    seen = set()
    for ind_A, ind_B in zip(modA_feats, modB_feats):
        if ind_A not in seen:
            seen.add(ind_A)
            val = corr_vals[ind_B]
            new_vals.append(val)
    new_mean_corr = sum(new_vals) / len(new_vals)
    # print(new_mean_corr)
    return new_mean_corr

## sim fns

In [8]:
import functools
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(
    R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad"
) -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def sim_random_baseline(
    rep1: torch.Tensor, rep2: torch.Tensor, sim_func: Callable, n_permutations: int = 10
) -> Dict[str, Any]:
    torch.manual_seed(1234)
    scores = []
    for _ in range(n_permutations):
        perm = torch.randperm(rep1.size(0))

        score = sim_func(rep1[perm, :], rep2)
        score = score if isinstance(score, float) else score["score"]

        scores.append(score)

    return {"baseline_scores": np.array(scores)}


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray], Dict[str, Any]],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray) -> Dict[str, Any]:
        for preprocess_func in self.preprocess_funcs:
            R = preprocess_func(R)
            Rp = preprocess_func(Rp)
        return self.similarity_func(R, Rp)

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return (
                func.__name__
                if not isinstance(func, functools.partial)
                else func.func.__name__
            )

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )

In [9]:
from typing import List, Set, Union

import numpy as np
import numpy.typing as npt
import sklearn.neighbors
import torch

# from llmcomp.measures.utils import to_numpy_if_needed


def _jac_sim_i(idx_R: Set[int], idx_Rp: Set[int]) -> float:
    return len(idx_R.intersection(idx_Rp)) / len(idx_R.union(idx_Rp))


def jaccard_similarity(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    k: int = 10,
    inner: str = "cosine",
    n_jobs: int = 8,
) -> float:
    R, Rp = to_numpy_if_needed(R, Rp)

    indices_R = nn_array_to_setlist(top_k_neighbors(R, k, inner, n_jobs))
    indices_Rp = nn_array_to_setlist(top_k_neighbors(Rp, k, inner, n_jobs))

    return float(
        np.mean(
            [_jac_sim_i(idx_R, idx_Rp) for idx_R, idx_Rp in zip(indices_R, indices_Rp)]
        )
    )


def top_k_neighbors(
    R: npt.NDArray,
    k: int,
    inner: str,
    n_jobs: int,
) -> npt.NDArray:
    # k+1 nearest neighbors, because we pass in all the data, which means that a point
    # will be the nearest neighbor to itself. We remove this point from the results and
    # report only the k nearest neighbors distinct from the point itself.
    nns = sklearn.neighbors.NearestNeighbors(
        n_neighbors=k + 1, metric=inner, n_jobs=n_jobs
    )
    nns.fit(R)
    _, nns = nns.kneighbors(R)
    return nns[:, 1:]


def nn_array_to_setlist(nn: npt.NDArray) -> List[Set[int]]:
    return [set(idx) for idx in nn]

In [10]:
import functools
import logging
from abc import ABC
from abc import abstractmethod
from dataclasses import dataclass
from dataclasses import field
from typing import Any
from typing import Callable
from typing import get_args
from typing import List
from typing import Literal
from typing import Optional
from typing import Protocol
from typing import Tuple
from typing import Union

import numpy as np
import numpy.typing as npt
import torch
from einops import rearrange
# from loguru import logger

log = logging.getLogger(__name__)


SHAPE_TYPE = Literal["nd", "ntd", "nchw"]

ND_SHAPE, NTD_SHAPE, NCHW_SHAPE = get_args(SHAPE_TYPE)[0], get_args(SHAPE_TYPE)[1], get_args(SHAPE_TYPE)[2]


class SimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float: ...


class RSMSimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE, n_jobs: int
    ) -> float: ...


@dataclass
class BaseSimilarityMeasure(ABC):
    larger_is_more_similar: bool
    is_symmetric: bool

    is_metric: bool | None = None
    invariant_to_affine: bool | None = None
    invariant_to_invertible_linear: bool | None = None
    invariant_to_ortho: bool | None = None
    invariant_to_permutation: bool | None = None
    invariant_to_isotropic_scaling: bool | None = None
    invariant_to_translation: bool | None = None
    name: str = field(init=False)

    def __post_init__(self):
        self.name = self.__class__.__name__

    @abstractmethod
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError


class FunctionalSimilarityMeasure(BaseSimilarityMeasure):
    @abstractmethod
    def __call__(self, output_a: torch.Tensor | npt.NDArray, output_b: torch.Tensor | npt.NDArray) -> float:
        raise NotImplementedError


@dataclass(kw_only=True)
class RepresentationalSimilarityMeasure(BaseSimilarityMeasure):
    sim_func: SimilarityFunction

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float:
        return self.sim_func(R, Rp, shape)


class RSMSimilarityMeasure(RepresentationalSimilarityMeasure):
    sim_func: RSMSimilarityFunction

    @staticmethod
    def estimate_good_number_of_jobs(R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray) -> int:
        # RSMs in are NxN (or DxD) so the number of jobs should roughly scale quadratically with increase in N (or D).
        # False! As long as sklearn-native metrics are used, they will use parallel implementations regardless of job
        # count. Each job would spawn their own threads, which leads to oversubscription of cores and thus slowdown.
        # This seems to be not fully correct (n_jobs=2 seems to actually use two cores), but using n_jobs=1 seems the
        # fastest.
        return 1

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
        n_jobs: Optional[int] = None,
    ) -> float:
        if n_jobs is None:
            n_jobs = self.estimate_good_number_of_jobs(R, Rp)
        return self.sim_func(R, Rp, shape, n_jobs=n_jobs)


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad") -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def normalize_row_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord=2, axis=1, keepdims=True)


def standardize(R: npt.NDArray) -> npt.NDArray:
    return (R - R.mean(axis=0, keepdims=True)) / R.std(axis=0)


def double_center(x: npt.NDArray) -> npt.NDArray:
    return x - x.mean(axis=0, keepdims=True) - x.mean(axis=1, keepdims=True) + x.mean()


def align_spatial_dimensions(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Aligns spatial representations by resizing them to the smallest spatial dimension.
    Subsequent aligned spatial representations are flattened, with the spatial aligned representations
    moving into the *sample* dimension.
    """
    R_re, Rp_re = resize_wh_reps(R, Rp)
    R_re = rearrange(R_re, "n c h w -> (n h w) c")
    Rp_re = rearrange(Rp_re, "n c h w -> (n h w) c")
    if R_re.shape[0] > 5000:
        logger.info(f"Got {R_re.shape[0]} samples in N after flattening. Subsampling to reduce compute.")
        subsample = R_re.shape[0] // 5000
        R_re = R_re[::subsample]
        Rp_re = Rp_re[::subsample]

    return R_re, Rp_re


def average_pool_downsample(R, resize: bool, new_size: tuple[int, int]):
    if not resize:
        return R  # do nothing
    else:
        is_numpy = isinstance(R, np.ndarray)
        R_torch = torch.from_numpy(R) if is_numpy else R
        R_torch = torch.nn.functional.adaptive_avg_pool2d(R_torch, new_size)
        return R_torch.numpy() if is_numpy else R_torch


def resize_wh_reps(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Function for resizing spatial representations that are not the same size.
    Does through fourier transform and resizing.

    Args:
        R: numpy array of shape  [batch_size, height, width, num_channels]
        RP: numpy array of shape [batch_size, height, width, num_channels]

    Returns:
        fft_acts1: numpy array of shape [batch_size, (new) height, (new) width, num_channels]
        fft_acts2: numpy array of shape [batch_size, (new) height, (new) width, num_channels]

    """
    height1, width1 = R.shape[2], R.shape[3]
    height2, width2 = Rp.shape[2], Rp.shape[3]
    if height1 != height2 or width1 != width2:
        height = min(height1, height2)
        width = min(width1, width2)
        new_size = [height, width]
        resize = True
    else:
        height = height1
        width = width1
        new_size = None
        resize = False

    # resize and preprocess with fft
    avg_ds1 = average_pool_downsample(R, resize=resize, new_size=new_size)
    avg_ds2 = average_pool_downsample(Rp, resize=resize, new_size=new_size)
    return avg_ds1, avg_ds2


def fft_resize(images, resize=False, new_size=None):
    """Function for applying DFT and resizing.

    This function takes in an array of images, applies the 2-d fourier transform
    and resizes them according to new_size, keeping the frequencies that overlap
    between the two sizes.

    Args:
              images: a numpy array with shape
                      [batch_size, height, width, num_channels]
              resize: boolean, whether or not to resize
              new_size: a tuple (size, size), with height and width the same

    Returns:
              im_fft_downsampled: a numpy array with shape
                           [batch_size, (new) height, (new) width, num_channels]
    """
    assert len(images.shape) == 4, "expecting images to be" "[batch_size, height, width, num_channels]"
    if resize:
        # FFT --> remove high frequencies --> inverse FFT
        im_complex = images.astype("complex64")
        im_fft = np.fft.fft2(im_complex, axes=(1, 2))
        im_shifted = np.fft.fftshift(im_fft, axes=(1, 2))

        center_width = im_shifted.shape[2] // 2
        center_height = im_shifted.shape[1] // 2
        half_w = new_size[0] // 2
        half_h = new_size[1] // 2
        cropped_fft = im_shifted[
            :, center_height - half_h : center_height + half_h, center_width - half_w : center_width + half_w, :
        ]
        cropped_fft_shifted_back = np.fft.ifft2(cropped_fft, axes=(1, 2))
        return cropped_fft_shifted_back.real
    else:
        return images


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray, SHAPE_TYPE], float],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray, shape: SHAPE_TYPE) -> float:
        try:
            for preprocess_func in self.preprocess_funcs:
                R = preprocess_func(R)
                Rp = preprocess_func(Rp)
            return self.similarity_func(R, Rp, shape)
        except ValueError as e:
            log.info(f"Pipeline failed: {e}")
            return np.nan

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return func.__name__ if not isinstance(func, functools.partial) else func.func.__name__

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )


def flatten(*args: Union[torch.Tensor, npt.NDArray], shape: SHAPE_TYPE) -> List[Union[torch.Tensor, npt.NDArray]]:
    if shape == "ntd":
        return list(map(flatten_nxtxd_to_ntxd, args))
    elif shape == "nd":
        return list(args)
    elif shape == "nchw":
        return list(map(flatten_nxcxhxw_to_nxchw, args))  # Flattening non-trivial for nchw
    else:
        raise ValueError("Unknown shape of representations. Must be one of 'ntd', 'nchw', 'nd'.")


def flatten_nxtxd_to_ntxd(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.flatten(R, start_dim=0, end_dim=1)
    log.debug("Shape after flattening: %s", str(R.shape))
    return R


def flatten_nxcxhxw_to_nxchw(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.reshape(R, (R.shape[0], -1))
    log.debug("Shape after flattening: %s", str(R.shape))
    return R

In [11]:
import scipy.optimize

def permutation_procrustes(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    optimal_permutation_alignment: Optional[Tuple[npt.NDArray, npt.NDArray]] = None,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    R, Rp = adjust_dimensionality(R, Rp)

    if not optimal_permutation_alignment:
        PR, PRp = scipy.optimize.linear_sum_assignment(R.T @ Rp, maximize=True)  # returns column assignments
        optimal_permutation_alignment = (PR, PRp)
    PR, PRp = optimal_permutation_alignment
    return float(np.linalg.norm(R[:, PR] - Rp[:, PRp], ord="fro"))

In [12]:
from typing import Optional
from typing import Union

import numpy as np
import numpy.typing as npt
import scipy.spatial.distance
import scipy.stats
import sklearn.metrics
import torch
# from repsim.measures.utils import flatten
# from repsim.measures.utils import RSMSimilarityMeasure
# from repsim.measures.utils import SHAPE_TYPE
# from repsim.measures.utils import to_numpy_if_needed


def representational_similarity_analysis(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    inner="correlation",
    outer="spearman",
    n_jobs: Optional[int] = None,
) -> float:
    """Representational similarity analysis

    Args:
        R (Union[torch.Tensor, npt.NDArray]): N x D representation
        Rp (Union[torch.Tensor, npt.NDArray]): N x D' representation
        inner (str, optional): inner similarity function for RSM. Must be one of
            scipy.spatial.distance.pdist identifiers . Defaults to "correlation".
        outer (str, optional): outer similarity function that compares RSMs. Defaults to
             "spearman". Must be one of "spearman", "euclidean"

    Returns:
        float: _description_
    """
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)

    if inner == "correlation":
        # n_jobs only works if metric is in PAIRWISE_DISTANCES as defined in sklearn, i.e., not for correlation.
        # But correlation = 1 - cosine dist of row-centered data, so we use the faster cosine metric and center the data.
        R = R - R.mean(axis=1, keepdims=True)
        S = scipy.spatial.distance.squareform(  # take the lower triangle of RSM
            1 - sklearn.metrics.pairwise_distances(R, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
        Rp = Rp - Rp.mean(axis=1, keepdims=True)
        Sp = scipy.spatial.distance.squareform(
            1 - sklearn.metrics.pairwise_distances(Rp, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
    elif inner == "euclidean":
        # take the lower triangle of RSM
        S = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(R, metric=inner, n_jobs=n_jobs), checks=False
        )
        Sp = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(Rp, metric=inner, n_jobs=n_jobs), checks=False
        )
    else:
        raise NotImplementedError(f"{inner=}")

    if outer == "spearman":
        return scipy.stats.spearmanr(S, Sp).statistic  # type:ignore
    elif outer == "euclidean":
        return float(np.linalg.norm(S - Sp, ord=2))
    else:
        raise ValueError(f"Unknown outer similarity function: {outer}")


class RSA(RSMSimilarityMeasure):
    def __init__(self):
        # choice of inner/outer in __call__ if fixed to default values, so these values are always the same
        super().__init__(
            sim_func=representational_similarity_analysis,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

In [13]:
##################################################################################
# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The core code for applying Canonical Correlation Analysis to deep networks.

This module contains the core functions to apply canonical correlation analysis
to deep neural networks. The main function is get_cca_similarity, which takes in
two sets of activations, typically the neurons in two layers and their outputs
on all of the datapoints D = [d_1,...,d_m] that have been passed through.

Inputs have shape (num_neurons1, m), (num_neurons2, m). This can be directly
applied used on fully connected networks. For convolutional layers, the 3d block
of neurons can either be flattened entirely, along channels, or alternatively,
the dft_ccas (Discrete Fourier Transform) module can be used.

See:
https://arxiv.org/abs/1706.05806
https://arxiv.org/abs/1806.05759
for full details.

"""
import numpy as np
# from repsim.measures.utils import align_spatial_dimensions

num_cca_trials = 5


def positivedef_matrix_sqrt(array):
    """Stable method for computing matrix square roots, supports complex matrices.

    Args:
              array: A numpy 2d array, can be complex valued that is a positive
                     definite symmetric (or hermitian) matrix

    Returns:
              sqrtarray: The matrix square root of array
    """
    w, v = np.linalg.eigh(array)
    #  A - np.dot(v, np.dot(np.diag(w), v.T))
    wsqrt = np.sqrt(w)
    sqrtarray = np.dot(v, np.dot(np.diag(wsqrt), np.conj(v).T))
    return sqrtarray


def remove_small(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon):
    """Takes covariance between X, Y, and removes values of small magnitude.

    Args:
              sigma_xx: 2d numpy array, variance matrix for x
              sigma_xy: 2d numpy array, crossvariance matrix for x,y
              sigma_yx: 2d numpy array, crossvariance matrixy for x,y,
                        (conjugate) transpose of sigma_xy
              sigma_yy: 2d numpy array, variance matrix for y
              epsilon : cutoff value for norm below which directions are thrown
                         away

    Returns:
              sigma_xx_crop: 2d array with low x norm directions removed
              sigma_xy_crop: 2d array with low x and y norm directions removed
              sigma_yx_crop: 2d array with low x and y norm directiosn removed
              sigma_yy_crop: 2d array with low y norm directions removed
              x_idxs: indexes of sigma_xx that were removed
              y_idxs: indexes of sigma_yy that were removed
    """

    x_diag = np.abs(np.diagonal(sigma_xx))
    y_diag = np.abs(np.diagonal(sigma_yy))
    x_idxs = x_diag >= epsilon
    y_idxs = y_diag >= epsilon

    sigma_xx_crop = sigma_xx[x_idxs][:, x_idxs]
    sigma_xy_crop = sigma_xy[x_idxs][:, y_idxs]
    sigma_yx_crop = sigma_yx[y_idxs][:, x_idxs]
    sigma_yy_crop = sigma_yy[y_idxs][:, y_idxs]

    return (sigma_xx_crop, sigma_xy_crop, sigma_yx_crop, sigma_yy_crop, x_idxs, y_idxs)


def compute_ccas(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon, verbose=True):
    """Main cca computation function, takes in variances and crossvariances.

    This function takes in the covariances and cross covariances of X, Y,
    preprocesses them (removing small magnitudes) and outputs the raw results of
    the cca computation, including cca directions in a rotated space, and the
    cca correlation coefficient values.

    Args:
              sigma_xx: 2d numpy array, (num_neurons_x, num_neurons_x)
                        variance matrix for x
              sigma_xy: 2d numpy array, (num_neurons_x, num_neurons_y)
                        crossvariance matrix for x,y
              sigma_yx: 2d numpy array, (num_neurons_y, num_neurons_x)
                        crossvariance matrix for x,y (conj) transpose of sigma_xy
              sigma_yy: 2d numpy array, (num_neurons_y, num_neurons_y)
                        variance matrix for y
              epsilon:  small float to help with stabilizing computations
              verbose:  boolean on whether to print intermediate outputs

    Returns:
              [ux, sx, vx]: [numpy 2d array, numpy 1d array, numpy 2d array]
                            ux and vx are (conj) transposes of each other, being
                            the canonical directions in the X subspace.
                            sx is the set of canonical correlation coefficients-
                            how well corresponding directions in vx, Vy correlate
                            with each other.
              [uy, sy, vy]: Same as above, but for Y space
              invsqrt_xx:   Inverse square root of sigma_xx to transform canonical
                            directions back to original space
              invsqrt_yy:   Same as above but for sigma_yy
              x_idxs:       The indexes of the input sigma_xx that were pruned
                            by remove_small
              y_idxs:       Same as above but for sigma_yy
    """

    (sigma_xx, sigma_xy, sigma_yx, sigma_yy, x_idxs, y_idxs) = remove_small(
        sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon
    )

    numx = sigma_xx.shape[0]
    numy = sigma_yy.shape[0]

    if numx == 0 or numy == 0:
        return (
            [0, 0, 0],
            [0, 0, 0],
            np.zeros_like(sigma_xx),
            np.zeros_like(sigma_yy),
            x_idxs,
            y_idxs,
        )

    if verbose:
        print("adding eps to diagonal and taking inverse")
    sigma_xx += epsilon * np.eye(numx)
    sigma_yy += epsilon * np.eye(numy)
    inv_xx = np.linalg.pinv(sigma_xx)
    inv_yy = np.linalg.pinv(sigma_yy)

    if verbose:
        print("taking square root")
    invsqrt_xx = positivedef_matrix_sqrt(inv_xx)
    invsqrt_yy = positivedef_matrix_sqrt(inv_yy)

    if verbose:
        print("dot products...")
    arr = np.dot(invsqrt_xx, np.dot(sigma_xy, invsqrt_yy))

    if verbose:
        print("trying to take final svd")
    u, s, v = np.linalg.svd(arr)

    if verbose:
        print("computed everything!")

    return [u, np.abs(s), v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs


def sum_threshold(array, threshold):
    """Computes threshold index of decreasing nonnegative array by summing.

    This function takes in a decreasing array nonnegative floats, and a
    threshold between 0 and 1. It returns the index i at which the sum of the
    array up to i is threshold*total mass of the array.

    Args:
              array: a 1d numpy array of decreasing, nonnegative floats
              threshold: a number between 0 and 1

    Returns:
              i: index at which np.sum(array[:i]) >= threshold
    """
    assert (threshold >= 0) and (threshold <= 1), "print incorrect threshold"

    for i in range(len(array)):
        if np.sum(array[:i]) / np.sum(array) >= threshold:
            return i


def create_zero_dict(compute_dirns, dimension):
    """Outputs a zero dict when neuron activation norms too small.

    This function creates a return_dict with appropriately shaped zero entries
    when all neuron activations are very small.

    Args:
              compute_dirns: boolean, whether to have zero vectors for directions
              dimension: int, defines shape of directions

    Returns:
              return_dict: a dict of appropriately shaped zero entries
    """
    return_dict = {}
    return_dict["mean"] = (np.asarray(0), np.asarray(0))
    return_dict["sum"] = (np.asarray(0), np.asarray(0))
    return_dict["cca_coef1"] = np.asarray(0)
    return_dict["cca_coef2"] = np.asarray(0)
    return_dict["idx1"] = 0
    return_dict["idx2"] = 0

    if compute_dirns:
        return_dict["cca_dirns1"] = np.zeros((1, dimension))
        return_dict["cca_dirns2"] = np.zeros((1, dimension))

    return return_dict


def get_cca_similarity(
    acts1,
    acts2,
    epsilon=0.0,
    threshold=0.98,
    compute_coefs=True,
    compute_dirns=False,
    verbose=True,
):
    """The main function for computing cca similarities.

    This function computes the cca similarity between two sets of activations,
    returning a dict with the cca coefficients, a few statistics of the cca
    coefficients, and (optionally) the actual directions.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              epsilon: small float to help stabilize computations

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.


              compute_coefs: boolean value determining whether coefficients
                             over neurons are computed. Needed for computing
                             directions

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

              verbose: Boolean, whether intermediate outputs are printed

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    # assert dimensionality equal
    assert acts1.shape[1] == acts2.shape[1], "dimensions don't match"
    # check that acts1, acts2 are transposition
    assert acts1.shape[0] < acts1.shape[1], "input must be number of neurons" "by datapoints"
    return_dict = {}

    # compute covariance with numpy function for extra stability
    numx = acts1.shape[0]
    numy = acts2.shape[0]

    covariance = np.cov(acts1, acts2)
    sigmaxx = covariance[:numx, :numx]
    sigmaxy = covariance[:numx, numx:]
    sigmayx = covariance[numx:, :numx]
    sigmayy = covariance[numx:, numx:]

    # rescale covariance to make cca computation more stable
    xmax = np.max(np.abs(sigmaxx))
    ymax = np.max(np.abs(sigmayy))
    sigmaxx /= xmax
    sigmayy /= ymax
    sigmaxy /= np.sqrt(xmax * ymax)
    sigmayx /= np.sqrt(xmax * ymax)

    ([u, s, v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs) = compute_ccas(
        sigmaxx, sigmaxy, sigmayx, sigmayy, epsilon=epsilon, verbose=verbose
    )

    # if x_idxs or y_idxs is all false, return_dict has zero entries
    if (not np.any(x_idxs)) or (not np.any(y_idxs)):
        return create_zero_dict(compute_dirns, acts1.shape[1])

    if compute_coefs:
        # also compute full coefficients over all neurons
        x_mask = np.dot(x_idxs.reshape((-1, 1)), x_idxs.reshape((1, -1)))
        y_mask = np.dot(y_idxs.reshape((-1, 1)), y_idxs.reshape((1, -1)))

        return_dict["coef_x"] = u.T
        return_dict["invsqrt_xx"] = invsqrt_xx
        return_dict["full_coef_x"] = np.zeros((numx, numx))
        np.place(return_dict["full_coef_x"], x_mask, return_dict["coef_x"])
        return_dict["full_invsqrt_xx"] = np.zeros((numx, numx))
        np.place(return_dict["full_invsqrt_xx"], x_mask, return_dict["invsqrt_xx"])

        return_dict["coef_y"] = v
        return_dict["invsqrt_yy"] = invsqrt_yy
        return_dict["full_coef_y"] = np.zeros((numy, numy))
        np.place(return_dict["full_coef_y"], y_mask, return_dict["coef_y"])
        return_dict["full_invsqrt_yy"] = np.zeros((numy, numy))
        np.place(return_dict["full_invsqrt_yy"], y_mask, return_dict["invsqrt_yy"])

        # compute means
        neuron_means1 = np.mean(acts1, axis=1, keepdims=True)
        neuron_means2 = np.mean(acts2, axis=1, keepdims=True)
        return_dict["neuron_means1"] = neuron_means1
        return_dict["neuron_means2"] = neuron_means2

    if compute_dirns:
        # orthonormal directions that are CCA directions
        cca_dirns1 = (
            np.dot(
                np.dot(return_dict["full_coef_x"], return_dict["full_invsqrt_xx"]),
                (acts1 - neuron_means1),
            )
            + neuron_means1
        )
        cca_dirns2 = (
            np.dot(
                np.dot(return_dict["full_coef_y"], return_dict["full_invsqrt_yy"]),
                (acts2 - neuron_means2),
            )
            + neuron_means2
        )

    # get rid of trailing zeros in the cca coefficients
    idx1 = sum_threshold(s, threshold)
    idx2 = sum_threshold(s, threshold)

    return_dict["cca_coef1"] = s
    return_dict["cca_coef2"] = s
    return_dict["x_idxs"] = x_idxs
    return_dict["y_idxs"] = y_idxs
    # summary statistics
    return_dict["mean"] = (np.mean(s[:idx1]), np.mean(s[:idx2]))
    return_dict["sum"] = (np.sum(s), np.sum(s))

    if compute_dirns:
        return_dict["cca_dirns1"] = cca_dirns1
        return_dict["cca_dirns2"] = cca_dirns2

    return return_dict


def robust_cca_similarity(acts1, acts2, threshold=0.98, epsilon=1e-6, compute_dirns=True):
    """Calls get_cca_similarity multiple times while adding noise.

    This function is very similar to get_cca_similarity, and can be used if
    get_cca_similarity doesn't converge for some pair of inputs. This function
    adds some noise to the activations to help convergence.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.

              epsilon: small float to help stabilize computations

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    for trial in range(num_cca_trials):
        try:
            return_dict = get_cca_similarity(acts1, acts2, threshold, compute_dirns)
        except np.linalg.LinAlgError:
            acts1 = acts1 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            acts2 = acts2 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            if trial + 1 == num_cca_trials:
                raise

    return return_dict
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py


def top_k_pca_comps(singular_values, threshold=0.99):
    total_variance = np.sum(singular_values**2)
    explained_variance = (singular_values**2) / total_variance
    cumulative_variance = np.cumsum(explained_variance)
    return np.argmax(cumulative_variance >= threshold * total_variance) + 1


def _svcca_original(acts1, acts2):
    # Copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    # Modification: get_cca_similarity is in the same file.
    # Modification: top-k PCA component selection s.t. explained variance > 0.99 total variance
    # Mean subtract activations
    cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
    cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

    # Perform SVD
    U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

    # top-k PCA components only
    k1 = top_k_pca_comps(s1)
    k2 = top_k_pca_comps(s2)

    svacts1 = np.dot(s1[:k1] * np.eye(k1), V1[:k1])
    # can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
    svacts2 = np.dot(s2[:k2] * np.eye(k2), V2[:k2])
    # can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

    svcca_results = get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    return np.mean(svcca_results["cca_coef1"])


# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py
# Modification: get_cca_similarity is in the same file.
def compute_pwcca(acts1, acts2, epsilon=0.0):
    """Computes projection weighting for weighting CCA coefficients

    Args:
         acts1: 2d numpy array, shaped (neurons, num_datapoints)
         acts2: 2d numpy array, shaped (neurons, num_datapoints)

    Returns:
         Original cca coefficient mean and weighted mean

    """
    sresults = get_cca_similarity(
        acts1,
        acts2,
        epsilon=epsilon,
        compute_dirns=False,
        compute_coefs=True,
        verbose=False,
    )
    if np.sum(sresults["x_idxs"]) <= np.sum(sresults["y_idxs"]):
        dirns = (
            np.dot(
                sresults["coef_x"],
                (acts1[sresults["x_idxs"]] - sresults["neuron_means1"][sresults["x_idxs"]]),
            )
            + sresults["neuron_means1"][sresults["x_idxs"]]
        )
        coefs = sresults["cca_coef1"]
        acts = acts1
        idxs = sresults["x_idxs"]
    else:
        dirns = (
            np.dot(
                sresults["coef_y"],
                (acts1[sresults["y_idxs"]] - sresults["neuron_means2"][sresults["y_idxs"]]),
            )
            + sresults["neuron_means2"][sresults["y_idxs"]]
        )
        coefs = sresults["cca_coef2"]
        acts = acts2
        idxs = sresults["y_idxs"]
    P, _ = np.linalg.qr(dirns.T)
    weights = np.sum(np.abs(np.dot(P.T, acts[idxs].T)), axis=1)
    weights = weights / np.sum(weights)

    return np.sum(weights * coefs), weights, coefs
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py


##################################################################################

from typing import Union  # noqa:e402

import numpy.typing as npt  # noqa:e402
import torch  # noqa:e402

# from repsim.measures.utils import (
#     SHAPE_TYPE,
#     flatten,
#     resize_wh_reps,
#     to_numpy_if_needed,
#     RepresentationalSimilarityMeasure,
# )  # noqa:e402


def svcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return _svcca_original(R.T, Rp.T)


def pwcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return compute_pwcca(R.T, Rp.T)[0]


class SVCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=svcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=True,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)


class PWCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=pwcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=False,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=False,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)

## get rand

In [14]:
def score_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modA_feats = np.random.choice(range(weight_matrix_np.shape[0]), size=num_feats, replace=False).tolist()
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    return sum(all_rand_scores) / len(all_rand_scores)

In [15]:
def score_rand_corr(num_runs, weight_matrix_np, weight_matrix_2, num_feats, highest_correlations_indices_AB, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()
            rand_modA_feats = [highest_correlations_indices_AB[index] for index in rand_modB_feats]

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    # print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return all_rand_scores

In [16]:
import random
def shuffle_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    for i in range(num_runs):
        row_idxs = list(range(num_feats))
        random.shuffle(row_idxs)
        if shapereq_bool:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs], "nd")
        else:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs])
        all_rand_scores.append(score)
    # return sum(all_rand_scores) / len(all_rand_scores)
    return all_rand_scores

## plot fns

In [17]:
def plot_svcca_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['svcca_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('SVCCA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [18]:
def plot_rsa_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['rsa_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['rsa_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('RSA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [19]:
def plot_meanCorr_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr'] for i in range(0, 12)]
    # unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    # rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    # label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [20]:
def plot_meanCorr_filt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [21]:
def plot_numFeats_afterFilt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['num_feat_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Num Feats Kept')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [22]:
# def plot_js_byLayer(layer_to_dictscores):
#     for key, sub_dict in layer_to_dictscores.items():
#         for sub_key, value in sub_dict.items():
#             sub_dict[sub_key] = round(value, 4)

#     layers = [f'L{i}' for i in range(0, 12)]
#     paired_values = [layer_to_dictscores[i]['1-1 jaccard_paired'] for i in range(0, 12)]
#     unpaired_values = [layer_to_dictscores[i]['1-1 jaccard_unpaired'] for i in range(0, 12)]

#     # Plotting configuration
#     x = np.arange(len(layers))  # label locations
#     width = 0.35  # width of the bars

#     # Increase figure size
#     fig, ax = plt.subplots(figsize=(12, 7))  # Slightly increased figure size
#     rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
#     rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

#     # Adding labels, title and custom x-axis tick labels
#     ax.set_ylabel('Jaccard NN')
#     ax.set_title('SAEs comparison by Pythia 70m MLP0 vs 160m MLP Layers')
#     ax.set_xticks(x)
#     ax.set_xticklabels(layers)
#     ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
#     ax.legend()

#     # Label bars with increased font size and different positioning for paired and unpaired
#     def label_bars(rects, is_paired):
#         for rect in rects:
#             height = rect.get_height()
#             label_height = height + 0.05 if is_paired else height + 0.01
#             ax.text(rect.get_x() + rect.get_width()/2., label_height,
#                     f'{height:.3f}',
#                     ha='center', va='bottom', fontsize=9)  # Increased font size to 12

#     label_bars(rects1, True)   # Paired bars
#     label_bars(rects2, False)  # Unpaired bars

#     # Adjust layout to prevent cutting off labels
#     plt.tight_layout()

#     # Increase y-axis limit to accommodate higher labels
#     ax.set_ylim(0, 1.1)  # Increased from 1 to 1.1

#     plt.show()

## interpret fns

In [23]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
): # -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [24]:
def store_top_toks(top_acts_indices, top_acts_values, batch_tokens):
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        new_str_token = tokenizer.decode(batch_tokens[batch_idx, seq_idx]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
        feat_samps.append(new_str_token)
    return feat_samps

In [25]:
def find_indices_with_keyword(fList, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    index_list = []
    for index, split_list in enumerate(fList):
        no_space_list = [i.replace(' ', '').lower() for i in split_list]
        for tok in no_space_list:
            if keyword.lower() == tok:
                index_list.append(index)
        # if keyword in no_space_list:
            # index_list.append(index)
    return index_list

In [26]:
from rich import print as rprint
def display_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'{batch_idx}\n'
        s += f'batchID: {batch_idx}, '
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = tokenizer.decode([batch_tokens[batch_idx, i].item()]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # Print the sequence, and the activation value
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'

    rprint(s)

In [27]:
# def store_top_seqs(top_acts_indices, top_acts_values, batch_tokens):
#     feat_samps = []
#     for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
#         # Get the sequence as a string (with some padding on either side of our sequence)
#         seq_start = max(seq_idx - 5, 0)
#         seq_end = min(seq_idx + 5, batch_tokens.shape[1])
#         seq = ""
#         # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
#         for i in range(seq_start, seq_end):
#             # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
#             new_str_token = tokenizer.decode([batch_tokens[batch_idx, i].item()]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
#             # if i == seq_idx:
#             #     new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
#             seq += new_str_token
#         feat_samps.append(seq)
#     return feat_samps

In [28]:
# def find_indices_with_keyword_bySeqs(fList_seqs, keyword):
#     feat_list = []
#     for feat_ind, top_seqs_lst in enumerate(fList_seqs):
#         for seq in top_seqs_lst:
#             split_list = seq.split(' ')
#             flag = False
#             for tok in split_list:
#                 if keyword.lower() == tok:
#                     feat_list.append(feat_ind)
#                     flag = True
#                     break
#             if flag:
#                 break
#     return feat_list

In [29]:
# def store_top_seqs(top_acts_indices, top_acts_values, batch_tokens):
#     feat_samps = []
#     for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
#         # Get the sequence with padding
#         seq_start = max(seq_idx - 2, 0)
#         seq_end = min(seq_idx + 2, batch_tokens.shape[1])
#         seq = ""
#         tokens = []
#         for i in range(seq_start, seq_end):
#             token_id = batch_tokens[batch_idx, i].item()
#             new_str_token = tokenizer.decode([token_id]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
#             tokens.append((new_str_token, i))
#             seq += new_str_token
#         # Store the sequence, tokens, and seq_idx
#         feat_samps.append((seq, tokens, seq_idx))
#     return feat_samps

# def get_word(tokens, idx_in_tokens):
#     # Initialize the word with the token at seq_idx
#     word_tokens = [tokens[idx_in_tokens][0]]
#     # Move backwards to find the start of the word
#     i = idx_in_tokens - 1
#     while i >= 0:
#         token_str, _ = tokens[i]
#         if token_str.startswith(' '):
#             break
#         word_tokens.insert(0, token_str)
#         i -= 1
#     # Move forwards to find the end of the word
#     i = idx_in_tokens + 1
#     while i < len(tokens):
#         token_str, _ = tokens[i]
#         if token_str.startswith(' '):
#             break
#         word_tokens.append(token_str)
#         i += 1
#     # Reconstruct the word and remove any spaces
#     word = ''.join(word_tokens).replace(' ', '')
#     return word


# def find_indices_with_keyword_bySeqs(fList_seqs, keyword):
#     feat_list = []
#     for feat_ind, top_seqs_lst in enumerate(fList_seqs):
#         for seq, tokens, seq_idx in top_seqs_lst:
#             # Find the position of seq_idx in tokens
#             idx_in_tokens = None
#             for i, (token_str, idx) in enumerate(tokens):
#                 if idx == seq_idx:
#                     idx_in_tokens = i
#                     break
#             if idx_in_tokens is None:
#                 continue  # seq_idx not found in tokens

#             # Reconstruct the word containing seq_idx
#             word = get_word(tokens, idx_in_tokens)

#             # Compare the reconstructed word with the keyword
#             if word.lower() == keyword.lower():
#                 feat_list.append(feat_ind)
#                 break  # Proceed to the next feature index
#     return feat_list


In [30]:
def store_top_seqs(top_acts_indices, top_acts_values, batch_tokens):
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 2, 0)
        seq_end = min(seq_idx + 2, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = tokenizer.decode([batch_tokens[batch_idx, i].item()]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                topTok = new_str_token
            seq += new_str_token
        feat_samps.append( (seq, topTok) )
    return feat_samps

In [31]:
def find_indices_with_keyword_bySeqs(fList_seqs, keyword):
    feat_list = []
    for feat_ind, top_seqs_andToks_lst in enumerate(fList_seqs):
        for top_seqs_andToks in top_seqs_andToks_lst:
            seq = top_seqs_andToks[0]
            topTok = top_seqs_andToks[1].replace(' ', '').lower()
            if keyword.lower() != topTok:
                continue
            split_list = seq.split(' ')
            flag = False
            for word in split_list:
                word = word.replace('.', '').replace(',', '').replace('?', '').replace('!', '').replace('\\n','')

                if keyword.lower() == word.lower():
                    feat_list.append(feat_ind)
                    flag = True
                    break
            if flag:
                break
    return feat_list

## get concept space features

In [32]:
def get_mixed_feats(fList_model_B, corr_inds, keywords):
    mixed_modA_feats = []
    mixed_modB_feats = []
    added_modA_feats = set()  # To track which modA feats have been added
    added_modB_feats = set()  # To track which modB feats have been added

    for kw in keywords:
        modB_feats = find_indices_with_keyword(fList_model_B, kw)
        for index in modB_feats:
            modA_feat = corr_inds[index]
            modB_feat = index

            # Check if the feature has already been added to maintain uniqueness
            if modA_feat not in added_modA_feats and modB_feat not in added_modB_feats:
                mixed_modA_feats.append(modA_feat)
                mixed_modB_feats.append(modB_feat)
                added_modA_feats.add(modA_feat)
                added_modB_feats.add(modB_feat)

    print("Unique modA feats: ", len(mixed_modA_feats))
    print("Unique modB feats: ", len(mixed_modB_feats))
    return mixed_modA_feats, mixed_modB_feats

In [33]:
def get_mixed_feats_with_kwList(fList_model_B, corr_inds, keywords):
    mixed_modA_feats = []
    mixed_modB_feats = []
    added_modA_feats = set()  # To track which modA feats have been added
    added_modB_feats = set()  # To track which modB feats have been added
    keywords_to_feats = {kw: 0 for kw in keywords} # kw : count

    for kw in keywords:
        modB_feats = find_indices_with_keyword(fList_model_B, kw)
        for index in modB_feats:
            modA_feat = corr_inds[index]
            modB_feat = index

            # Check if the feature has already been added to maintain uniqueness
            if modA_feat not in added_modA_feats and modB_feat not in added_modB_feats:
                mixed_modA_feats.append(modA_feat)
                mixed_modB_feats.append(modB_feat)
                added_modA_feats.add(modA_feat)
                added_modB_feats.add(modB_feat)
                keywords_to_feats[kw] += 1

    print("Unique modA feats: ", len(mixed_modA_feats))
    print("Unique modB feats: ", len(mixed_modB_feats))
    return mixed_modA_feats, mixed_modB_feats, keywords_to_feats

## get actv fns

In [34]:
# def get_weights_and_acts(name, cfg_dict, layer_id, outputs):
def get_weights_and_acts(name, layer_id, outputs):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    hookpoint = "layers." + str(layer_id)

    sae = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

    weight_matrix = sae.W_dec.cpu().detach().numpy()

    with torch.inference_mode():
        # reshaped_activations_A = sae.pre_acts(outputs.to("cuda"))
        reshaped_activations = sae.pre_acts(outputs.hidden_states[layer_id].to("cuda"))

    first_dim_reshaped = reshaped_activations.shape[0] * reshaped_activations.shape[1]
    reshaped_activations = reshaped_activations.reshape(first_dim_reshaped, reshaped_activations.shape[-1]).cpu()

    # return weight_matrix_np, reshaped_activations_A, orig
    return weight_matrix, reshaped_activations

In [35]:
def count_zero_columns(tensor):
    # Check if all elements in each column are zero
    zero_columns = np.all(tensor == 0, axis=0)
    # Count True values in the zero_columns array
    zero_cols_indices = np.where(zero_columns)[0]
    return np.sum(zero_columns), zero_cols_indices

## run expm fns

In [36]:
def run_expm(layer_id, outputs, outputs_2):
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 512}
    weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, cfg_dict, layer_id, outputs)
    # zero_cols_count, zero_cols_indices = count_zero_columns(reshaped_activations_A.cpu().numpy())
    # print("Number of zero columns:", zero_cols_count) #, zero_cols_indices

    name = "EleutherAI/sae-pythia-160m-32k"
    for layerID_2 in range(0, 12): # 0, 12
        dictscores = {}

        # redef
        cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 768}
        weight_matrix_2, reshaped_activations_B = get_weights_and_acts(name, cfg_dict, layerID_2, outputs_2)


        highest_correlations_indices_AB, highest_correlations_values_AB = batched_correlation(reshaped_activations_A, reshaped_activations_B)

        num_unq_pairs = len(list(set(highest_correlations_indices_AB)))
        print("% unique: ", num_unq_pairs / len(highest_correlations_indices_AB))

        dictscores["mean_actv_corr"] = sum(highest_correlations_values_AB) / len(highest_correlations_values_AB)

        ###########
        # filter

        sorted_feat_counts = Counter(highest_correlations_indices_AB).most_common()
        # kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 100]
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 1]

        filt_corr_ind_A = []
        filt_corr_ind_B = []
        seen = set()
        for ind_B, ind_A in enumerate(highest_correlations_indices_AB):
            if ind_A in kept_modA_feats:
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
            elif ind_A not in seen:  # only keep one if it's over count X
                seen.add(ind_A)
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
        # num_unq_pairs = len(list(set(filt_corr_ind_A)))
        # print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
        # print("num feats after filt: ", len(filt_corr_ind_A))

        new_highest_correlations_indices_A = []
        new_highest_correlations_indices_B = []
        new_highest_correlations_values = []

        for ind_A, ind_B in zip(filt_corr_ind_A, filt_corr_ind_B):
            val = highest_correlations_values_AB[ind_B]
            if val > 0:
                new_highest_correlations_indices_A.append(ind_A)
                new_highest_correlations_indices_B.append(ind_B)
                new_highest_correlations_values.append(val)

        # print("% unique after rmv 0s: ", num_unq_pairs / len(new_highest_correlations_indices_A))
        # print("num feats after rmv 0s: ", len(new_highest_correlations_indices_A))
        dictscores["num_feat_kept"] = len(new_highest_correlations_indices_A)

        dictscores["mean_actv_corr_filt"] = sum(new_highest_correlations_values) / len(new_highest_correlations_values)

        ###########
        # sim tests

        # # num_feats = len(filt_corr_ind_A)
        num_feats = len(new_highest_correlations_indices_A)

        # dictscores["svcca_paired"] = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
        dictscores["svcca_paired"] = svcca(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")

        dictscores["svcca_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                                  svcca, shapereq_bool=True)

        dictscores["rsa_paired"] = representational_similarity_analysis(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")
        dictscores["rsa_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                                  representational_similarity_analysis, shapereq_bool=True)

        print("Layer: " + str(layerID_2))
        for key, value in dictscores.items():
            print(key + ": " + str(value))
        print("\n")

        layer_to_dictscores[layerID_2] = dictscores
    return layer_to_dictscores

# load data

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
from datasets import load_dataset
# dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [39]:
batch_size = 100
maxseqlen = 300

def get_next_batch(dataset_iter, batch_size=100):
    batch = []
    for _ in range(batch_size):
        try:
            sample = next(dataset_iter)
            batch.append(sample['text'])
        except StopIteration:
            break
    return batch

dataset_iter = iter(dataset)
batch = get_next_batch(dataset_iter, batch_size)

# Tokenize the batch
inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=maxseqlen)

# load models

In [40]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
model_2 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

## get LLM actvs

In [41]:
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    outputs_2 = model_2(**inputs, output_hidden_states=True)

# concept keywords

In [42]:
keywords = {}

# keywords['Time'] = [
#     "day", "night", "week", "month", "year", "hour", "minute", "second", "now", "soon",
#     "later", "early", "late", "morning", "evening", "noon", "midnight", "dawn", "dusk", "past",
#     "present", "future", "before", "after", "yesterday", "today", "tomorrow", "next", "previous", "soon",
#     "instant", "era", "age", "decade", "century", "millennium",
#     "moment", "pause", "wait", "begin", "start", "end", "finish", "stop", "continue",
#     "forever", "constant", "frequent",
#     "occasion", "season", "spring", "summer", "autumn", "fall", "winter", "anniversary", "deadline", "schedule",
#     "calendar", "clock", "duration", "interval", "epoch", "generation", "period", "cycle", "timespan",
#     "shift", "quarter", "term", "phase", "lifetime", "century", "minute", "timeline", "delay",
#     "prompt", "timely", "recurrent", "daily", "weekly", "monthly", "yearly", "annual", "biweekly", "timeframe"
# ]

# keywords['Calendar'] = [
#     "day", "night", "week", "month", "year", "hour", "minute", "second",
#     "morning", "evening", "noon", "midnight", "dawn", "dusk",
#     "yesterday", "today", "tomorrow",
#     "decade", "century", "millennium",
#     "season", "spring", "summer", "autumn", "fall", "winter",
#     "calendar", "clock",
#     "century", "minute",
#     "daily", "weekly", "monthly", "yearly", "annual", "biweekly", "timeframe"
# ]

# keywords['People/Roles'] = [
#                 "man", "girl", "boy", "kid", "dad", "mom", "son", "sis", "bro",
#                 "chief", "priest", "king", "queen", "duke", "lord", "friend", "clerk", "coach",
#                 "nurse", "doc", "maid", "clown", "guest", "peer",
#                 "punk", "nerd", "jock", "chief"
# ]

# keywords['Nature'] = [
#     "tree", "grass", "stone", "rock", "cliff", "hill",
#     "dirt", "sand", "mud", "wind", "storm", "rain", "cloud", "sun",
#     "moon", "leaf", "branch", "twig", "root", "bark", "seed",
#     "tide", "lake", "pond", "creek", "sea", "wood", "field",
#     "shore", "snow", "ice", "flame", "fire", "fog", "dew", "hail",
#     "sky", "earth", "glade", "cave", "peak", "ridge", "dust", "air",
#     "mist", "heat"
# ]

# keywords['Emotions'] = [
#     "joy", "glee", "pride", "grief", "fear", "hope", "love", "hate", "pain", "shame",
#     "bliss", "rage", "calm", "shock", "dread", "guilt", "peace", "trust", "scorn", "doubt",
#     "hurt", "wrath", "laugh", "cry", "smile", "frown", "gasp", "blush", "sigh", "grin",
#     "woe", "spite", "envy", "glow", "thrill", "mirth", "bored", "cheer", "charm", "grace",
#     "shy", "brave", "proud", "glad", "mad", "sad", "tense", "free", "kind"
# ]

# keywords['MonthNames'] = [
#     "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
# ]

# keywords['Countries'] = [
#     "USA", "Canada", "Brazil", "Mexico", "Germany", "France", "Italy", "Spain", "UK", "Australia",
#     "China", "Japan", "India", "Russia", "Korea", "Argentina", "Egypt", "Iran", "Turkey"
# ]

keywords['Biology'] = [
    "gene", "DNA", "RNA", "virus", "bacteria", "fungus",
    "brain", "lung", "blood", "bone", "skin", "muscle", "nerve", "vein", "organ",
    "evolve", "enzyme", "protein", "lipid", "membrane", "antibody", "antigen",
    "ligand", "substrate", "receptor", "cell", "chromosome", "nucleus", "cytoplasm"
]

# get labels

## A: all layers

In [ ]:
model_A_layer_labels = {} # layerID : fList_model_A_seqs
modeltype = 'pythia70m'
name = "EleutherAI/sae-pythia-70m-32k"

for layer_id in range(0, 6):
    print(layer_id)
    hookpoint = "layers." + str(layer_id)
    sae = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

    weight_matrix_np = sae.W_dec.cpu().detach().numpy()

    with torch.inference_mode():
        outputs = model(**inputs, output_hidden_states=True)
        feature_acts_A = sae.pre_acts(outputs.hidden_states[layer_id].to('cuda'))

    first_dim_reshaped = feature_acts_A.shape[0] * feature_acts_A.shape[1]
    reshaped_activations_A = feature_acts_A.reshape(first_dim_reshaped, feature_acts_A.shape[-1]).cpu()

    # store feature : lst of top strs
    fList_model_A_seqs = []
    samp_m = 5

    for feature_idx in range(feature_acts_A.shape[-1]):
        if feature_idx % 5000 == 0:
            print('Feature: ', feature_idx)
        ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_A, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
        fList_model_A_seqs.append(store_top_seqs(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

    model_A_layer_labels[layer_id] = fList_model_A_seqs
    # with open(f'fList_L{layer_id}_{modeltype}.pkl', 'wb') as f:
    #     pickle.dump(fList_model_A_seqs, f)
    # files.download(f'fList_L{layer_id}_{modeltype}.pkl')



In [ ]:
with open(f'fList_allLayers_{modeltype}.pkl', 'wb') as f:
    pickle.dump(model_A_layer_labels, f)
files.download(f'fList_allLayers_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### load

In [ ]:
with open(f'fList_allLayers_pythia70m.pkl', 'rb') as f:
    model_A_layer_labels = pickle.load(f)

## B

In [ ]:
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
layer_id_2 = 9

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
# store feature : lst of top strs
fList_model_B_seqs = []
samp_m = 5

for feature_idx in range(feature_acts_B.shape[-1]):
# for feature_idx in range(5):
    if feature_idx % 5000 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_B, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B_seqs.append(store_top_seqs(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  5000
Feature:  10000
Feature:  15000
Feature:  20000
Feature:  25000
Feature:  30000


In [ ]:
import pickle
from google.colab import files
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(fList_model_B_seqs, f)
files.download(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

# run expm fn

In [43]:
# def run_semantic_subspace_expms(fList_model_B_seqs, fList_model_A_seqs, weight_matrix_np, reshaped_activations_A, outputs):
def run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs):
    metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
    # map_type_list = ['manyTo1'] # , '1To1'
    map_type_list = ['1To1']
    layer_to_dictscores = {}


    name = "EleutherAI/sae-pythia-70m-32k"
    for layerID_2 in range(1, 6):
        print("Layer: " + str(layerID_2))
        dictscores = {}

        weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, layerID_2, outputs)

        all_scores_dict = {}
        for concept_name in keywords.keys():
            print(concept_name)
            scores_dict = {}

            fList_model_A_seqs = model_A_layer_labels[layerID_2]

            new_keywords = keywords[concept_name]

            mixed_modA_feats = set()
            mixed_modB_feats = set()
            for kw in new_keywords:
                modB_feats = find_indices_with_keyword_bySeqs(fList_model_B_seqs, kw)
                modA_feats = find_indices_with_keyword_bySeqs(fList_model_A_seqs, kw)
                mixed_modA_feats.update(modA_feats)
                mixed_modB_feats.update(modB_feats)
            mixed_modA_feats = list(mixed_modA_feats)
            mixed_modB_feats = list(mixed_modB_feats)

            if len(mixed_modA_feats) <= 2 or len(mixed_modB_feats)  <= 2:
                for map_type in map_type_list:
                    for metric_name, metric_fn in metric_dict.items():
                        scores_dict[f'num_unq_pairs'] = None
                        scores_dict[f'paired_{metric_name}_{map_type}'] = None
                        # scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = None
                        # scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  None
                all_scores_dict[concept_name] = scores_dict
                continue

            subset_inds, subset_vals = batched_correlation(reshaped_activations_A[:, mixed_modA_feats],
                                               reshaped_activations_B[:, mixed_modB_feats])

            # num_unq_pairs = len(list(set(subset_inds)))
            # print("% unique: ", num_unq_pairs / len(subset_inds))
            # print(num_unq_pairs)
            # sum(subset_vals) / len(subset_vals)

            # filt_corr_ind_A, filt_corr_ind_B, num_unq_pairs = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

            subset_sorted_feat_counts = Counter(subset_inds).most_common()
            subset_kept_modA_feats = [feat_ID for feat_ID, count in subset_sorted_feat_counts if count == 1]

            filt_corr_ind_A = []
            filt_corr_ind_B = []
            seen = set()
            for ind_B, ind_A in enumerate(subset_inds):
                if ind_A in subset_kept_modA_feats:
                    filt_corr_ind_A.append(ind_A)
                    filt_corr_ind_B.append(ind_B)
                elif ind_A not in seen:  # only keep one if it's over count X
                    seen.add(ind_A)
                    filt_corr_ind_A.append(ind_A)
                    filt_corr_ind_B.append(ind_B)
            num_unq_pairs = len(list(set(filt_corr_ind_A)))
            # print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
            scores_dict[f'num_unq_pairs'] = num_unq_pairs

            original_A_indices = [mixed_modA_feats[ind] for ind in filt_corr_ind_A]
            original_B_indices = [mixed_modB_feats[i] for i in filt_corr_ind_B]

            if len(original_A_indices)  <= 2 or len(original_B_indices)  <= 2:
                for map_type in map_type_list:
                    for metric_name, metric_fn in metric_dict.items():
                        scores_dict[f'num_unq_pairs'] = None
                        scores_dict[f'paired_{metric_name}_{map_type}'] = None
                        # scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = None
                        # scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  None
                all_scores_dict[concept_name] = scores_dict
                continue

            for map_type in map_type_list:
                # print(map_type)
                if map_type == '1To1':
                    X_subset = weight_matrix_np[original_A_indices]
                    Y_subset = weight_matrix_2[original_B_indices]
                    num_feats = len(list(set(filt_corr_ind_A)))
                # else:
                #     X_subset = weight_matrix_np[mixed_modA_feats]
                #     Y_subset = weight_matrix_2[mixed_modB_feats]
                #     num_feats = len(list(set(mixed_modA_feats)))

                # scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

                for metric_name, metric_fn in metric_dict.items():
                    try:
                        paired_score = metric_fn(X_subset, Y_subset, "nd")
                    except:
                        pdb.set_trace()
                    scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

                    # rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                    #                 num_feats, subset_inds, metric_fn, True)
                    # scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
                    # scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

                    if len(original_A_indices) > 100:
                        num_runs = 100
                    else:
                        num_runs = 1000
                    rand_shuff_scores = shuffle_rand(num_runs, X_subset, Y_subset, Y_subset.shape[0],
                                                    metric_fn, shapereq_bool=True)
                    # scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
                    # scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

            all_scores_dict[concept_name] = scores_dict

        for key, value in all_scores_dict.items():
            print(key + ": " + str(value))
        print("\n")

        layer_to_dictscores[layerID_2] = all_scores_dict
    return layer_to_dictscores

# loop for any layer pair: L3

## run

In [ ]:
layer_id_2 = 3
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layers.3/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs)

Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 239, 'paired_SVCCA_1To1': 0.28998390205836444, 'rand_shuff_mean_SVCCA_1To1': 0.050484335939896784, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.06550371831706248, 'rand_shuff_mean_RSA_1To1': -1.788960648735946e-05, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 116, 'paired_SVCCA_1To1': 0.3762321590384507, 'rand_shuff_mean_SVCCA_1To1': 0.06953198758356226, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.05260770833739177, 'rand_shuff_mean_RSA_1To1': -0.00039805096754677653, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 24, 'paired_SVCCA_1To1': 0.06421575601800202, 'rand_shuff_mean_SVCCA_1To1': 0.16721339978970498, 'rand_shuff_pval_SVCCA_1To1': 0.769, 'paired_RSA_1To1': 0.1118431529230462, 'rand_shuff_mean_RSA_1To1': 0.004361626689116875, 'rand_shuff_pval_RSA_1To1': 0.167}
Nature: {'num_unq_pairs': 30, 'paired_SVCCA_1To1': 0.02378756312271985, 'rand

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

...

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 323, 'paired_SVCCA_1To1': 0.20524250088056062, 'rand_shuff_mean_SVCCA_1To1': 0.04507110554163397, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.015428395983483707, 'rand_shuff_mean_RSA_1To1': 0.000631925867922469, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 222, 'paired_SVCCA_1To1': 0.022452491930582987, 'rand_shuff_mean_SVCCA_1To1': 0.05631198349241587, 'rand_shuff_pval_SVCCA_1To1': 0.71, 'paired_RSA_1To1': 0.004062779319802674, 'rand_shuff_mean_RSA_1To1': 0.0005897578694970457, 'rand_shuff_pval_RSA_1To1': 0.33}
People/Roles: {'num_unq_pairs': 16, 'paired_SVCCA_1To1': 0.03140248195046895, 'rand_shuff_mean_SVCCA_1To1': 0.2006223418831324, 'rand_shuff_pval_SVCCA_1To1': 0.883, 'paired_RSA_1To1': -0.01555663587749149, 'rand_shuff_mean_RSA_1To1': 0.0047658795749704825, 'rand_shuff_pval_RSA_1To1': 0.547}
Nature: {'num_unq_pairs': 18, 'paired_SVCCA_1To1': 0.27896968191924143, 'r

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 175, 'paired_SVCCA_1To1': 0.0889572900637649, 'rand_shuff_mean_SVCCA_1To1': 0.0705167097630801, 'rand_shuff_pval_SVCCA_1To1': 0.3, 'paired_RSA_1To1': 0.012770866444743342, 'rand_shuff_mean_RSA_1To1': 3.0626736327443274e-05, 'rand_shuff_pval_RSA_1To1': 0.05}
Calendar: {'num_unq_pairs': 91, 'paired_SVCCA_1To1': 0.09068527907362907, 'rand_shuff_mean_SVCCA_1To1': 0.08323984419224263, 'rand_shuff_pval_SVCCA_1To1': 0.394, 'paired_RSA_1To1': 0.002752894495848722, 'rand_shuff_mean_RSA_1To1': 0.0005204492433743921, 'rand_shuff_pval_RSA_1To1': 0.436}
People/Roles: {'num_unq_pairs': 15, 'paired_SVCCA_1To1': 0.5950847097899448, 'rand_shuff_mean_SVCCA_1To1': 0.2103374900703021, 'rand_shuff_pval_SVCCA_1To1': 0.02, 'paired_RSA_1To1': 0.10944432925565001, 'rand_shuff_mean_RSA_1To1': 0.006252674683806762, 'rand_shuff_pval_RSA_1To1': 0.207}
Nature: {'num_unq_pairs': 18, 'paired_SVCCA_1To1': 0.2644828334276832, 'rand_

In [ ]:
layer_to_dictscores

{1: {'Time': {'num_unq_pairs': 239,
   'paired_SVCCA_1To1': 0.28998390205836444,
   'rand_shuff_mean_SVCCA_1To1': 0.050484335939896784,
   'rand_shuff_pval_SVCCA_1To1': 0.0,
   'paired_RSA_1To1': 0.06550371831706248,
   'rand_shuff_mean_RSA_1To1': -1.788960648735946e-05,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'Calendar': {'num_unq_pairs': 116,
   'paired_SVCCA_1To1': 0.3762321590384507,
   'rand_shuff_mean_SVCCA_1To1': 0.06953198758356226,
   'rand_shuff_pval_SVCCA_1To1': 0.0,
   'paired_RSA_1To1': 0.05260770833739177,
   'rand_shuff_mean_RSA_1To1': -0.00039805096754677653,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'People/Roles': {'num_unq_pairs': 24,
   'paired_SVCCA_1To1': 0.06421575601800202,
   'rand_shuff_mean_SVCCA_1To1': 0.16721339978970498,
   'rand_shuff_pval_SVCCA_1To1': 0.769,
   'paired_RSA_1To1': 0.1118431529230462,
   'rand_shuff_mean_RSA_1To1': 0.004361626689116875,
   'rand_shuff_pval_RSA_1To1': 0.167},
  'Nature': {'num_unq_pairs': 30,
   'paired_SVCCA_1To1': 0.023787563

In [ ]:
with open(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L1

In [ ]:
all_scores_dict = layer_to_dictscores[1]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,116
Countries,20
Emotions,23
MonthNames,44
Nature,30
People/Roles,24
Time,239


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.29,0.05,0.00
Calendar,0.38,0.07,0.00
People/Roles,0.06,0.17,0.77
Nature,0.02,0.15,0.90
Emotions,0.10,0.17,0.65
MonthNames,0.49,0.12,0.00
Countries,0.54,0.19,0.02


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.07,-0.0,0.00
Calendar,0.05,-0.0,0.00
People/Roles,0.11,0.0,0.17
Nature,0.19,-0.0,0.03
Emotions,-0.10,-0.0,0.84
MonthNames,0.04,0.0,0.15
Countries,0.34,-0.0,0.01


## analyze data: L2

In [ ]:
all_scores_dict = layer_to_dictscores[2]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,179
Countries,18
Emotions,17
MonthNames,27
Nature,35
People/Roles,18
Time,272


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.18,0.04,0.00
Calendar,0.30,0.06,0.00
People/Roles,0.30,0.20,0.23
Nature,0.30,0.14,0.08
Emotions,0.02,0.21,0.96
MonthNames,0.49,0.16,0.02
Countries,0.50,0.20,0.04


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,0.00,0.00
Calendar,0.02,-0.00,0.01
People/Roles,0.29,-0.00,0.00
Nature,0.15,0.00,0.06
Emotions,-0.03,0.00,0.58
MonthNames,0.18,0.00,0.00
Countries,0.29,0.01,0.03


## analyze data: L3

In [ ]:
all_scores_dict = layer_to_dictscores[3]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,102
Countries,16
Emotions,16
MonthNames,34
Nature,29
People/Roles,22
Time,198


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.02,0.06,0.84
Calendar,0.19,0.08,0.05
People/Roles,0.51,0.18,0.02
Nature,0.46,0.15,0.01
Emotions,0.02,0.21,0.93
MonthNames,0.28,0.14,0.09
Countries,0.29,0.21,0.25


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.03,0.0,0.00
Calendar,0.01,-0.0,0.31
People/Roles,0.31,-0.0,0.00
Nature,0.28,-0.0,0.01
Emotions,0.19,0.0,0.08
MonthNames,0.02,0.0,0.30
Countries,0.14,-0.0,0.20


## analyze data: L4

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,222
Countries,8
Emotions,16
MonthNames,28
Nature,18
People/Roles,16
Time,323


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.21,0.05,0.00
Calendar,0.02,0.06,0.71
People/Roles,0.03,0.20,0.88
Nature,0.28,0.20,0.26
Emotions,0.08,0.21,0.78
MonthNames,0.15,0.15,0.45
Countries,0.29,0.30,0.45


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,0.00,0.00
Calendar,0.00,0.00,0.33
People/Roles,-0.02,0.00,0.55
Nature,0.08,-0.00,0.24
Emotions,-0.03,-0.00,0.60
MonthNames,-0.02,-0.00,0.68
Countries,-0.15,-0.01,0.81


## analyze data: L5

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,222
Countries,8
Emotions,16
MonthNames,28
Nature,18
People/Roles,16
Time,323


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.21,0.05,0.00
Calendar,0.02,0.06,0.71
People/Roles,0.03,0.20,0.88
Nature,0.28,0.20,0.26
Emotions,0.08,0.21,0.78
MonthNames,0.15,0.15,0.45
Countries,0.29,0.30,0.45


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,0.00,0.00
Calendar,0.00,0.00,0.33
People/Roles,-0.02,0.00,0.55
Nature,0.08,-0.00,0.24
Emotions,-0.03,-0.00,0.60
MonthNames,-0.02,-0.00,0.68
Countries,-0.15,-0.01,0.81


# loop for any layer pair: L5

## run

In [ ]:
layer_id_2 = 5
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs)

Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 259, 'paired_SVCCA_1To1': 0.46836613820772555, 'rand_shuff_mean_SVCCA_1To1': 0.04521067336882189, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.08168966182836748, 'rand_shuff_mean_RSA_1To1': -6.95488241551396e-06, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 126, 'paired_SVCCA_1To1': 0.551726333410265, 'rand_shuff_mean_SVCCA_1To1': 0.06622282136428839, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.10987484913731153, 'rand_shuff_mean_RSA_1To1': -0.0005415367193588258, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 33, 'paired_SVCCA_1To1': 0.25092094899333106, 'rand_shuff_mean_SVCCA_1To1': 0.1432941368133709, 'rand_shuff_pval_SVCCA_1To1': 0.156, 'paired_RSA_1To1': 0.1728194917324097, 'rand_shuff_mean_RSA_1To1': 0.003353544855513237, 'rand_shuff_pval_RSA_1To1': 0.009}
Nature: {'num_unq_pairs': 47, 'paired_SVCCA_1To1': 0.3113790687482951, 'rand_shuff

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 274, 'paired_SVCCA_1To1': 0.5649329550692975, 'rand_shuff_mean_SVCCA_1To1': 0.03845846459353617, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.08370806149568347, 'rand_shuff_mean_RSA_1To1': 0.00018963821472321396, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 163, 'paired_SVCCA_1To1': 0.6181021078968528, 'rand_shuff_mean_SVCCA_1To1': 0.06521611094476903, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.06351176623943536, 'rand_shuff_mean_RSA_1To1': -0.00102951332972668, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 22, 'paired_SVCCA_1To1': 0.24231049186197043, 'rand_shuff_mean_SVCCA_1To1': 0.18129696579032892, 'rand_shuff_pval_SVCCA_1To1': 0.284, 'paired_RSA_1To1': 0.06531377168558578, 'rand_shuff_mean_RSA_1To1': -0.00907326077221131, 'rand_shuff_pval_RSA_1To1': 0.234}
Nature: {'num_unq_pairs': 39, 'paired_SVCCA_1To1': 0.1582202838908822, 'rand_shuf

...

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 228, 'paired_SVCCA_1To1': 0.5895674735517556, 'rand_shuff_mean_SVCCA_1To1': 0.05545780573767359, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.09740988776228149, 'rand_shuff_mean_RSA_1To1': -0.00025711950282116604, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 126, 'paired_SVCCA_1To1': 0.649097380971248, 'rand_shuff_mean_SVCCA_1To1': 0.060824394710637775, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.09312356729795312, 'rand_shuff_mean_RSA_1To1': -0.000875319306997057, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 31, 'paired_SVCCA_1To1': 0.4710842687171789, 'rand_shuff_mean_SVCCA_1To1': 0.14687785624007896, 'rand_shuff_pval_SVCCA_1To1': 0.007, 'paired_RSA_1To1': 0.18214551695493802, 'rand_shuff_mean_RSA_1To1': 0.0012120955806881261, 'rand_shuff_pval_RSA_1To1': 0.016}
Nature: {'num_unq_pairs': 46, 'paired_SVCCA_1To1': 0.5004843507297421, 'rand_sh

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 282, 'paired_SVCCA_1To1': 0.008097791489968996, 'rand_shuff_mean_SVCCA_1To1': 0.04506199865530913, 'rand_shuff_pval_SVCCA_1To1': 0.87, 'paired_RSA_1To1': 0.03204180312786893, 'rand_shuff_mean_RSA_1To1': -0.00018365828206623293, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 187, 'paired_SVCCA_1To1': 0.09006604461573274, 'rand_shuff_mean_SVCCA_1To1': 0.06649550007381631, 'rand_shuff_pval_SVCCA_1To1': 0.3, 'paired_RSA_1To1': 0.023741374218231137, 'rand_shuff_mean_RSA_1To1': -0.0014118603821169438, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 26, 'paired_SVCCA_1To1': 0.2784749168707524, 'rand_shuff_mean_SVCCA_1To1': 0.1617290956194695, 'rand_shuff_pval_SVCCA_1To1': 0.173, 'paired_RSA_1To1': 0.04826805096743748, 'rand_shuff_mean_RSA_1To1': -0.002334729869085698, 'rand_shuff_pval_RSA_1To1': 0.193}
Nature: {'num_unq_pairs': 32, 'paired_SVCCA_1To1': 0.2659651173009545, 'ra

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 213, 'paired_SVCCA_1To1': 0.022393455749730628, 'rand_shuff_mean_SVCCA_1To1': 0.05287393614283203, 'rand_shuff_pval_SVCCA_1To1': 0.72, 'paired_RSA_1To1': 0.026887318426763226, 'rand_shuff_mean_RSA_1To1': 0.0009152178462946816, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 112, 'paired_SVCCA_1To1': 0.33424068814870755, 'rand_shuff_mean_SVCCA_1To1': 0.08630775388442123, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.008339901613492557, 'rand_shuff_mean_RSA_1To1': -0.000944306110991692, 'rand_shuff_pval_RSA_1To1': 0.29}
People/Roles: {'num_unq_pairs': 24, 'paired_SVCCA_1To1': 0.0928471840705162, 'rand_shuff_mean_SVCCA_1To1': 0.16634667228984784, 'rand_shuff_pval_SVCCA_1To1': 0.663, 'paired_RSA_1To1': 0.006035302007676832, 'rand_shuff_mean_RSA_1To1': 0.0017775984931721824, 'rand_shuff_pval_RSA_1To1': 0.473}
Nature: {'num_unq_pairs': 33, 'paired_SVCCA_1To1': 0.28807657954910093, '

In [ ]:
layer_to_dictscores

{1: {'Time': {'num_unq_pairs': 259,
   'paired_SVCCA_1To1': 0.46836613820772555,
   'rand_shuff_mean_SVCCA_1To1': 0.04521067336882189,
   'rand_shuff_pval_SVCCA_1To1': 0.0,
   'paired_RSA_1To1': 0.08168966182836748,
   'rand_shuff_mean_RSA_1To1': -6.95488241551396e-06,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'Calendar': {'num_unq_pairs': 126,
   'paired_SVCCA_1To1': 0.551726333410265,
   'rand_shuff_mean_SVCCA_1To1': 0.06622282136428839,
   'rand_shuff_pval_SVCCA_1To1': 0.0,
   'paired_RSA_1To1': 0.10987484913731153,
   'rand_shuff_mean_RSA_1To1': -0.0005415367193588258,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'People/Roles': {'num_unq_pairs': 33,
   'paired_SVCCA_1To1': 0.25092094899333106,
   'rand_shuff_mean_SVCCA_1To1': 0.1432941368133709,
   'rand_shuff_pval_SVCCA_1To1': 0.156,
   'paired_RSA_1To1': 0.1728194917324097,
   'rand_shuff_mean_RSA_1To1': 0.003353544855513237,
   'rand_shuff_pval_RSA_1To1': 0.009},
  'Nature': {'num_unq_pairs': 47,
   'paired_SVCCA_1To1': 0.31137906874829

In [ ]:
with open(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L1

In [ ]:
all_scores_dict = layer_to_dictscores[1]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,126
Countries,34
Emotions,24
MonthNames,36
Nature,47
People/Roles,33
Time,259


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.47,0.05,0.00
Calendar,0.55,0.07,0.00
People/Roles,0.25,0.14,0.16
Nature,0.31,0.12,0.03
Emotions,0.67,0.17,0.00
MonthNames,0.72,0.14,0.00
Countries,0.21,0.14,0.24


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.08,-0.0,0.00
Calendar,0.11,-0.0,0.00
People/Roles,0.17,0.0,0.01
Nature,0.18,-0.0,0.00
Emotions,0.46,-0.0,0.00
MonthNames,0.13,0.0,0.01
Countries,0.09,-0.0,0.08


## analyze data: L2

In [ ]:
all_scores_dict = layer_to_dictscores[2]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,163
Countries,22
Emotions,22
MonthNames,29
Nature,39
People/Roles,22
Time,274


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.56,0.04,0.00
Calendar,0.62,0.07,0.00
People/Roles,0.24,0.18,0.28
Nature,0.16,0.13,0.34
Emotions,0.53,0.18,0.02
MonthNames,0.42,0.15,0.03
Countries,0.49,0.18,0.01


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.08,0.00,0.00
Calendar,0.06,-0.00,0.00
People/Roles,0.07,-0.01,0.23
Nature,0.06,-0.00,0.25
Emotions,0.35,0.00,0.00
MonthNames,0.04,-0.00,0.23
Countries,0.19,-0.00,0.02


## analyze data: L3

In [ ]:
all_scores_dict = layer_to_dictscores[3]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,126
Countries,30
Emotions,24
MonthNames,32
Nature,46
People/Roles,31
Time,228


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.59,0.06,0.00
Calendar,0.65,0.06,0.00
People/Roles,0.47,0.15,0.01
Nature,0.50,0.12,0.00
Emotions,0.83,0.17,0.00
MonthNames,0.72,0.14,0.00
Countries,0.58,0.15,0.00


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.10,-0.0,0.00
Calendar,0.09,-0.0,0.00
People/Roles,0.18,0.0,0.02
Nature,0.22,0.0,0.00
Emotions,0.46,-0.0,0.00
MonthNames,0.10,0.0,0.03
Countries,0.34,0.0,0.00


## analyze data: L4

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,187
Countries,12
Emotions,18
MonthNames,24
Nature,32
People/Roles,26
Time,282


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.01,0.05,0.87
Calendar,0.09,0.07,0.30
People/Roles,0.28,0.16,0.17
Nature,0.27,0.15,0.14
Emotions,0.17,0.20,0.52
MonthNames,0.22,0.17,0.30
Countries,0.04,0.24,0.91


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.03,-0.0,0.00
Calendar,0.02,-0.0,0.00
People/Roles,0.05,-0.0,0.19
Nature,0.12,0.0,0.03
Emotions,0.10,0.0,0.13
MonthNames,0.08,-0.0,0.11
Countries,0.18,-0.0,0.08


## analyze data: L5

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,187
Countries,12
Emotions,18
MonthNames,24
Nature,32
People/Roles,26
Time,282


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.01,0.05,0.87
Calendar,0.09,0.07,0.30
People/Roles,0.28,0.16,0.17
Nature,0.27,0.15,0.14
Emotions,0.17,0.20,0.52
MonthNames,0.22,0.17,0.30
Countries,0.04,0.24,0.91


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.03,-0.0,0.00
Calendar,0.02,-0.0,0.00
People/Roles,0.05,-0.0,0.19
Nature,0.12,0.0,0.03
Emotions,0.10,0.0,0.13
MonthNames,0.08,-0.0,0.11
Countries,0.18,-0.0,0.08


# loop for any layer pair: L7

## run

In [ ]:
layer_id_2 = 7
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs)

Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 350, 'paired_SVCCA_1To1': 0.1551131691243702, 'rand_shuff_mean_SVCCA_1To1': 0.04138216061152415, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.017245082302684223, 'rand_shuff_mean_RSA_1To1': -3.438455225131842e-05, 'rand_shuff_pval_RSA_1To1': 0.01}
Calendar: {'num_unq_pairs': 156, 'paired_SVCCA_1To1': 0.16657328942949431, 'rand_shuff_mean_SVCCA_1To1': 0.060051677330574274, 'rand_shuff_pval_SVCCA_1To1': 0.04, 'paired_RSA_1To1': 0.03697755792185085, 'rand_shuff_mean_RSA_1To1': 0.0035002665407151048, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 33, 'paired_SVCCA_1To1': 0.28971860040606395, 'rand_shuff_mean_SVCCA_1To1': 0.14586068779041234, 'rand_shuff_pval_SVCCA_1To1': 0.11, 'paired_RSA_1To1': 0.17367471404271415, 'rand_shuff_mean_RSA_1To1': -0.0017410225414207516, 'rand_shuff_pval_RSA_1To1': 0.006}
Nature: {'num_unq_pairs': 54, 'paired_SVCCA_1To1': 0.39578028802806764, 'r

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 391, 'paired_SVCCA_1To1': 0.2971175848621547, 'rand_shuff_mean_SVCCA_1To1': 0.03952412393910098, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.02142853872417965, 'rand_shuff_mean_RSA_1To1': -0.00039794234525827274, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 224, 'paired_SVCCA_1To1': 0.40365009721862966, 'rand_shuff_mean_SVCCA_1To1': 0.0522414117194995, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.0357601904590857, 'rand_shuff_mean_RSA_1To1': -0.000309484827567197, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 26, 'paired_SVCCA_1To1': 0.2636818383816572, 'rand_shuff_mean_SVCCA_1To1': 0.15676159041382384, 'rand_shuff_pval_SVCCA_1To1': 0.172, 'paired_RSA_1To1': 0.19926119931319877, 'rand_shuff_mean_RSA_1To1': -0.0013931806926031447, 'rand_shuff_pval_RSA_1To1': 0.027}
Nature: {'num_unq_pairs': 42, 'paired_SVCCA_1To1': 0.12521609385514623, 'rand_s

...

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 394, 'paired_SVCCA_1To1': 0.13500818300489628, 'rand_shuff_mean_SVCCA_1To1': 0.042554358680845415, 'rand_shuff_pval_SVCCA_1To1': 0.04, 'paired_RSA_1To1': 0.05403893276268476, 'rand_shuff_mean_RSA_1To1': -0.0004968199073628296, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 185, 'paired_SVCCA_1To1': 0.5236302298068894, 'rand_shuff_mean_SVCCA_1To1': 0.05923274152177403, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.08640831903375337, 'rand_shuff_mean_RSA_1To1': -0.0020611050875758764, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 33, 'paired_SVCCA_1To1': 0.6972754946725805, 'rand_shuff_mean_SVCCA_1To1': 0.14141730705096434, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.18846048939947124, 'rand_shuff_mean_RSA_1To1': -0.0036321223861659913, 'rand_shuff_pval_RSA_1To1': 0.002}
Nature: {'num_unq_pairs': 60, 'paired_SVCCA_1To1': 0.35489475949264987, 'rand

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 411, 'paired_SVCCA_1To1': 0.23563486722834506, 'rand_shuff_mean_SVCCA_1To1': 0.03675780866462098, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.007898432265199776, 'rand_shuff_mean_RSA_1To1': 0.00022552540148011114, 'rand_shuff_pval_RSA_1To1': 0.08}
Calendar: {'num_unq_pairs': 273, 'paired_SVCCA_1To1': 0.056760851263306294, 'rand_shuff_mean_SVCCA_1To1': 0.048357668124784425, 'rand_shuff_pval_SVCCA_1To1': 0.35, 'paired_RSA_1To1': 0.013479914142982014, 'rand_shuff_mean_RSA_1To1': 0.0006129575866354736, 'rand_shuff_pval_RSA_1To1': 0.05}
People/Roles: {'num_unq_pairs': 53, 'paired_SVCCA_1To1': 0.058720109047754994, 'rand_shuff_mean_SVCCA_1To1': 0.11533385686812248, 'rand_shuff_pval_SVCCA_1To1': 0.684, 'paired_RSA_1To1': 0.04638625350524706, 'rand_shuff_mean_RSA_1To1': -8.390932832472664e-05, 'rand_shuff_pval_RSA_1To1': 0.059}
Nature: {'num_unq_pairs': 72, 'paired_SVCCA_1To1': 0.249710245752304

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 380, 'paired_SVCCA_1To1': 0.03189615560012538, 'rand_shuff_mean_SVCCA_1To1': 0.03676032637847411, 'rand_shuff_pval_SVCCA_1To1': 0.44, 'paired_RSA_1To1': 0.005199362215813994, 'rand_shuff_mean_RSA_1To1': -0.0012335597312066206, 'rand_shuff_pval_RSA_1To1': 0.11}
Calendar: {'num_unq_pairs': 190, 'paired_SVCCA_1To1': 0.11303805770892017, 'rand_shuff_mean_SVCCA_1To1': 0.06290983510267101, 'rand_shuff_pval_SVCCA_1To1': 0.12, 'paired_RSA_1To1': -0.008218026285108954, 'rand_shuff_mean_RSA_1To1': -0.0003239049952835511, 'rand_shuff_pval_RSA_1To1': 0.8}
People/Roles: {'num_unq_pairs': 37, 'paired_SVCCA_1To1': 0.3768731421293669, 'rand_shuff_mean_SVCCA_1To1': 0.13580004274654847, 'rand_shuff_pval_SVCCA_1To1': 0.017, 'paired_RSA_1To1': 0.036418185948683866, 'rand_shuff_mean_RSA_1To1': -0.00021871681513439887, 'rand_shuff_pval_RSA_1To1': 0.211}
Nature: {'num_unq_pairs': 59, 'paired_SVCCA_1To1': 0.249617300305072

In [ ]:
layer_to_dictscores

{1: {'Time': {'num_unq_pairs': 350,
   'paired_SVCCA_1To1': 0.1551131691243702,
   'rand_shuff_mean_SVCCA_1To1': 0.04138216061152415,
   'rand_shuff_pval_SVCCA_1To1': 0.0,
   'paired_RSA_1To1': 0.017245082302684223,
   'rand_shuff_mean_RSA_1To1': -3.438455225131842e-05,
   'rand_shuff_pval_RSA_1To1': 0.01},
  'Calendar': {'num_unq_pairs': 156,
   'paired_SVCCA_1To1': 0.16657328942949431,
   'rand_shuff_mean_SVCCA_1To1': 0.060051677330574274,
   'rand_shuff_pval_SVCCA_1To1': 0.04,
   'paired_RSA_1To1': 0.03697755792185085,
   'rand_shuff_mean_RSA_1To1': 0.0035002665407151048,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'People/Roles': {'num_unq_pairs': 33,
   'paired_SVCCA_1To1': 0.28971860040606395,
   'rand_shuff_mean_SVCCA_1To1': 0.14586068779041234,
   'rand_shuff_pval_SVCCA_1To1': 0.11,
   'paired_RSA_1To1': 0.17367471404271415,
   'rand_shuff_mean_RSA_1To1': -0.0017410225414207516,
   'rand_shuff_pval_RSA_1To1': 0.006},
  'Nature': {'num_unq_pairs': 54,
   'paired_SVCCA_1To1': 0.395780

In [ ]:
with open(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L1

In [ ]:
all_scores_dict = layer_to_dictscores[1]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,156
Countries,35
Emotions,29
MonthNames,40
Nature,54
People/Roles,33
Time,350


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.16,0.04,0.00
Calendar,0.17,0.06,0.04
People/Roles,0.29,0.15,0.11
Nature,0.40,0.11,0.00
Emotions,0.54,0.15,0.00
MonthNames,0.15,0.13,0.35
Countries,0.25,0.13,0.12


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.0,0.01
Calendar,0.04,0.0,0.00
People/Roles,0.17,-0.0,0.01
Nature,0.24,0.0,0.00
Emotions,0.15,-0.0,0.03
MonthNames,0.02,0.0,0.35
Countries,0.03,-0.0,0.31


## analyze data: L2

In [ ]:
all_scores_dict = layer_to_dictscores[2]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,224
Countries,23
Emotions,28
MonthNames,30
Nature,42
People/Roles,26
Time,391


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.30,0.04,0.00
Calendar,0.40,0.05,0.00
People/Roles,0.26,0.16,0.17
Nature,0.13,0.13,0.43
Emotions,0.42,0.16,0.02
MonthNames,0.12,0.15,0.54
Countries,0.04,0.18,0.84


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.0,0.00
Calendar,0.04,-0.0,0.00
People/Roles,0.20,-0.0,0.03
Nature,0.13,0.0,0.06
Emotions,0.08,0.0,0.14
MonthNames,0.04,0.0,0.18
Countries,0.01,-0.0,0.41


## analyze data: L3

In [ ]:
all_scores_dict = layer_to_dictscores[3]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,185
Countries,36
Emotions,26
MonthNames,39
Nature,60
People/Roles,33
Time,394


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.14,0.04,0.04
Calendar,0.52,0.06,0.00
People/Roles,0.70,0.14,0.00
Nature,0.35,0.11,0.01
Emotions,0.63,0.17,0.00
MonthNames,0.23,0.13,0.17
Countries,0.31,0.13,0.04


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.05,-0.0,0.00
Calendar,0.09,-0.0,0.00
People/Roles,0.19,-0.0,0.00
Nature,0.09,-0.0,0.02
Emotions,0.23,-0.0,0.00
MonthNames,0.06,0.0,0.10
Countries,0.19,0.0,0.01


## analyze data: L4

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,273
Countries,14
Emotions,29
MonthNames,32
Nature,72
People/Roles,53
Time,411


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.24,0.04,0.00
Calendar,0.06,0.05,0.35
People/Roles,0.06,0.12,0.68
Nature,0.25,0.10,0.04
Emotions,0.01,0.15,0.98
MonthNames,0.04,0.15,0.86
Countries,0.45,0.22,0.11


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,0.0,0.08
Calendar,0.01,0.0,0.05
People/Roles,0.05,-0.0,0.06
Nature,0.00,-0.0,0.44
Emotions,0.03,0.0,0.32
MonthNames,0.04,-0.0,0.19
Countries,0.14,0.0,0.13


## analyze data: L5

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,273
Countries,14
Emotions,29
MonthNames,32
Nature,72
People/Roles,53
Time,411


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.24,0.04,0.00
Calendar,0.06,0.05,0.35
People/Roles,0.06,0.12,0.68
Nature,0.25,0.10,0.04
Emotions,0.01,0.15,0.98
MonthNames,0.04,0.15,0.86
Countries,0.45,0.22,0.11


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,0.0,0.08
Calendar,0.01,0.0,0.05
People/Roles,0.05,-0.0,0.06
Nature,0.00,-0.0,0.44
Emotions,0.03,0.0,0.32
MonthNames,0.04,-0.0,0.19
Countries,0.14,0.0,0.13


# loop for any layer pair: L9

## run

In [ ]:
layer_id_2 = 9
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs)

Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 340, 'paired_SVCCA_1To1': 0.12185764361880548, 'rand_shuff_mean_SVCCA_1To1': 0.04462379833681247, 'rand_shuff_pval_SVCCA_1To1': 0.05, 'paired_RSA_1To1': 0.019532032808835424, 'rand_shuff_mean_RSA_1To1': -0.00010928542074538617, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 142, 'paired_SVCCA_1To1': 0.12960035848422896, 'rand_shuff_mean_SVCCA_1To1': 0.07465895553588307, 'rand_shuff_pval_SVCCA_1To1': 0.23, 'paired_RSA_1To1': 0.026662976747402086, 'rand_shuff_mean_RSA_1To1': -0.0003908857190611138, 'rand_shuff_pval_RSA_1To1': 0.01}
People/Roles: {'num_unq_pairs': 30, 'paired_SVCCA_1To1': 0.35084414822781246, 'rand_shuff_mean_SVCCA_1To1': 0.14926821464423562, 'rand_shuff_pval_SVCCA_1To1': 0.06, 'paired_RSA_1To1': 0.13196197318412622, 'rand_shuff_mean_RSA_1To1': 0.0025223444718684078, 'rand_shuff_pval_RSA_1To1': 0.024}
Nature: {'num_unq_pairs': 47, 'paired_SVCCA_1To1': 0.36908379634021754,

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 366, 'paired_SVCCA_1To1': 0.14547888301956513, 'rand_shuff_mean_SVCCA_1To1': 0.04403777522732485, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.02126286275510205, 'rand_shuff_mean_RSA_1To1': -6.203727616276489e-05, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 195, 'paired_SVCCA_1To1': 0.17640457603987952, 'rand_shuff_mean_SVCCA_1To1': 0.05031048293138433, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.022178837044166386, 'rand_shuff_mean_RSA_1To1': -0.002313192735161154, 'rand_shuff_pval_RSA_1To1': 0.02}
People/Roles: {'num_unq_pairs': 24, 'paired_SVCCA_1To1': 0.20273950223458254, 'rand_shuff_mean_SVCCA_1To1': 0.1708556823675368, 'rand_shuff_pval_SVCCA_1To1': 0.347, 'paired_RSA_1To1': 0.12300166949672522, 'rand_shuff_mean_RSA_1To1': -0.004433034916739201, 'rand_shuff_pval_RSA_1To1': 0.09}
Nature: {'num_unq_pairs': 44, 'paired_SVCCA_1To1': 0.10283498333106164, 'rand

...

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 367, 'paired_SVCCA_1To1': 0.36022340537185665, 'rand_shuff_mean_SVCCA_1To1': 0.03986056256190045, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.04482746263743215, 'rand_shuff_mean_RSA_1To1': 0.00017348680410799683, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 185, 'paired_SVCCA_1To1': 0.028236662838801586, 'rand_shuff_mean_SVCCA_1To1': 0.05339693351097984, 'rand_shuff_pval_SVCCA_1To1': 0.64, 'paired_RSA_1To1': 0.0390453103050831, 'rand_shuff_mean_RSA_1To1': 0.0005809275760153098, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 35, 'paired_SVCCA_1To1': 0.4011888246329234, 'rand_shuff_mean_SVCCA_1To1': 0.13933982493588673, 'rand_shuff_pval_SVCCA_1To1': 0.021, 'paired_RSA_1To1': 0.17348979046570837, 'rand_shuff_mean_RSA_1To1': 0.0001667490425090105, 'rand_shuff_pval_RSA_1To1': 0.004}
Nature: {'num_unq_pairs': 47, 'paired_SVCCA_1To1': 0.43801587130213304, 'rand

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 435, 'paired_SVCCA_1To1': 0.18376438322833236, 'rand_shuff_mean_SVCCA_1To1': 0.03876300231382161, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.016193063940773203, 'rand_shuff_mean_RSA_1To1': -0.0006631828274768811, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 279, 'paired_SVCCA_1To1': 0.23243237413541715, 'rand_shuff_mean_SVCCA_1To1': 0.052256947227100364, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.028882501141478393, 'rand_shuff_mean_RSA_1To1': 0.0004929586018939777, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 50, 'paired_SVCCA_1To1': 0.35465175066936705, 'rand_shuff_mean_SVCCA_1To1': 0.1160987913912055, 'rand_shuff_pval_SVCCA_1To1': 0.015, 'paired_RSA_1To1': 0.005879709152632846, 'rand_shuff_mean_RSA_1To1': 0.00030534527400739295, 'rand_shuff_pval_RSA_1To1': 0.43}
Nature: {'num_unq_pairs': 59, 'paired_SVCCA_1To1': 0.51358653791674, 'rand

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 375, 'paired_SVCCA_1To1': 0.09856303499004666, 'rand_shuff_mean_SVCCA_1To1': 0.04231442474546677, 'rand_shuff_pval_SVCCA_1To1': 0.07, 'paired_RSA_1To1': 0.020271277830487034, 'rand_shuff_mean_RSA_1To1': -0.00027974342458744457, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 181, 'paired_SVCCA_1To1': 0.1591516391327824, 'rand_shuff_mean_SVCCA_1To1': 0.06593580643399512, 'rand_shuff_pval_SVCCA_1To1': 0.05, 'paired_RSA_1To1': 0.02336189142832335, 'rand_shuff_mean_RSA_1To1': -0.00047222411333564543, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 36, 'paired_SVCCA_1To1': 0.527478418556751, 'rand_shuff_mean_SVCCA_1To1': 0.13603530204655573, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.05504926946150029, 'rand_shuff_mean_RSA_1To1': -0.0005288647861594623, 'rand_shuff_pval_RSA_1To1': 0.104}
Nature: {'num_unq_pairs': 42, 'paired_SVCCA_1To1': 0.2661685464176921, 'ran

In [ ]:
layer_to_dictscores

{1: {'Time': {'num_unq_pairs': 340,
   'paired_SVCCA_1To1': 0.12185764361880548,
   'rand_shuff_mean_SVCCA_1To1': 0.04462379833681247,
   'rand_shuff_pval_SVCCA_1To1': 0.05,
   'paired_RSA_1To1': 0.019532032808835424,
   'rand_shuff_mean_RSA_1To1': -0.00010928542074538617,
   'rand_shuff_pval_RSA_1To1': 0.0},
  'Calendar': {'num_unq_pairs': 142,
   'paired_SVCCA_1To1': 0.12960035848422896,
   'rand_shuff_mean_SVCCA_1To1': 0.07465895553588307,
   'rand_shuff_pval_SVCCA_1To1': 0.23,
   'paired_RSA_1To1': 0.026662976747402086,
   'rand_shuff_mean_RSA_1To1': -0.0003908857190611138,
   'rand_shuff_pval_RSA_1To1': 0.01},
  'People/Roles': {'num_unq_pairs': 30,
   'paired_SVCCA_1To1': 0.35084414822781246,
   'rand_shuff_mean_SVCCA_1To1': 0.14926821464423562,
   'rand_shuff_pval_SVCCA_1To1': 0.06,
   'paired_RSA_1To1': 0.13196197318412622,
   'rand_shuff_mean_RSA_1To1': 0.0025223444718684078,
   'rand_shuff_pval_RSA_1To1': 0.024},
  'Nature': {'num_unq_pairs': 47,
   'paired_SVCCA_1To1': 0.369

In [ ]:
with open(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L1

In [ ]:
all_scores_dict = layer_to_dictscores[1]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,142
Countries,38
Emotions,34
MonthNames,50
Nature,47
People/Roles,30
Time,340


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.12,0.04,0.05
Calendar,0.13,0.07,0.23
People/Roles,0.35,0.15,0.06
Nature,0.37,0.12,0.01
Emotions,0.41,0.14,0.01
MonthNames,0.13,0.12,0.38
Countries,0.31,0.13,0.05


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.0,0.00
Calendar,0.03,-0.0,0.01
People/Roles,0.13,0.0,0.02
Nature,0.19,0.0,0.00
Emotions,0.06,0.0,0.17
MonthNames,0.06,-0.0,0.03
Countries,0.07,0.0,0.06


## analyze data: L2

In [ ]:
all_scores_dict = layer_to_dictscores[2]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,195
Countries,22
Emotions,24
MonthNames,34
Nature,44
People/Roles,24
Time,366


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.15,0.04,0.00
Calendar,0.18,0.05,0.00
People/Roles,0.20,0.17,0.35
Nature,0.10,0.12,0.51
Emotions,0.31,0.17,0.14
MonthNames,0.18,0.14,0.29
Countries,0.20,0.17,0.37


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.00,0.00
Calendar,0.02,-0.00,0.02
People/Roles,0.12,-0.00,0.09
Nature,0.08,-0.00,0.11
Emotions,0.07,-0.01,0.17
MonthNames,-0.02,-0.00,0.64
Countries,0.10,0.00,0.12


## analyze data: L3

In [ ]:
all_scores_dict = layer_to_dictscores[3]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,185
Countries,38
Emotions,26
MonthNames,40
Nature,47
People/Roles,35
Time,367


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.36,0.04,0.00
Calendar,0.03,0.05,0.64
People/Roles,0.40,0.14,0.02
Nature,0.44,0.12,0.00
Emotions,0.40,0.17,0.06
MonthNames,0.13,0.13,0.41
Countries,0.45,0.13,0.00


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.04,0.0,0.00
Calendar,0.04,0.0,0.00
People/Roles,0.17,0.0,0.00
Nature,0.17,0.0,0.00
Emotions,0.11,0.0,0.09
MonthNames,0.04,-0.0,0.14
Countries,0.19,-0.0,0.00


## analyze data: L4

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,279
Countries,15
Emotions,30
MonthNames,39
Nature,59
People/Roles,50
Time,435


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.18,0.04,0.00
Calendar,0.23,0.05,0.00
People/Roles,0.35,0.12,0.02
Nature,0.51,0.11,0.00
Emotions,0.37,0.15,0.04
MonthNames,0.08,0.13,0.63
Countries,0.05,0.21,0.85


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.0,0.00
Calendar,0.03,0.0,0.00
People/Roles,0.01,0.0,0.43
Nature,0.09,-0.0,0.01
Emotions,0.05,-0.0,0.21
MonthNames,-0.00,-0.0,0.55
Countries,0.04,-0.0,0.31


## analyze data: L5

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,279
Countries,15
Emotions,30
MonthNames,39
Nature,59
People/Roles,50
Time,435


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.18,0.04,0.00
Calendar,0.23,0.05,0.00
People/Roles,0.35,0.12,0.02
Nature,0.51,0.11,0.00
Emotions,0.37,0.15,0.04
MonthNames,0.08,0.13,0.63
Countries,0.05,0.21,0.85


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.02,-0.0,0.00
Calendar,0.03,0.0,0.00
People/Roles,0.01,0.0,0.43
Nature,0.09,-0.0,0.01
Emotions,0.05,-0.0,0.21
MonthNames,-0.00,-0.0,0.55
Countries,0.04,-0.0,0.31


# loop for any layer pair: L11

## run

In [ ]:
layer_id_2 = 11
modeltype = 'pythia160m'
name = "EleutherAI/sae-pythia-160m-32k"
with open(f'fList_model_B_L{layer_id_2}_{modeltype}.pkl', 'rb') as f:
    fList_model_B_seqs = pickle.load(f)

In [ ]:
hookpoint = "layers." + str(layer_id_2)
sae_2 = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)

weight_matrix_2 = sae_2.W_dec.cpu().detach().numpy()

with torch.inference_mode():
    feature_acts_B = sae_2.pre_acts(outputs_2.hidden_states[layer_id_2].to("cuda"))

first_dim_reshaped = feature_acts_B.shape[0] * feature_acts_B.shape[1]
reshaped_activations_B = feature_acts_B.reshape(first_dim_reshaped, feature_acts_B.shape[-1]).cpu()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layers.11/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

In [ ]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_seqs, model_A_layer_labels, outputs)

Layer: 1


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

layers.1/cfg.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 277, 'paired_SVCCA_1To1': 0.04721582249661352, 'rand_shuff_mean_SVCCA_1To1': 0.05126210433138111, 'rand_shuff_pval_SVCCA_1To1': 0.46, 'paired_RSA_1To1': 0.013489543331351614, 'rand_shuff_mean_RSA_1To1': 0.0007914899435218007, 'rand_shuff_pval_RSA_1To1': 0.01}
Calendar: {'num_unq_pairs': 130, 'paired_SVCCA_1To1': 0.10704092405185818, 'rand_shuff_mean_SVCCA_1To1': 0.08042893493422304, 'rand_shuff_pval_SVCCA_1To1': 0.31, 'paired_RSA_1To1': 0.011259458409499375, 'rand_shuff_mean_RSA_1To1': 0.0012132195117975817, 'rand_shuff_pval_RSA_1To1': 0.18}
People/Roles: {'num_unq_pairs': 26, 'paired_SVCCA_1To1': 0.014760603647810696, 'rand_shuff_mean_SVCCA_1To1': 0.15932149387484248, 'rand_shuff_pval_SVCCA_1To1': 0.954, 'paired_RSA_1To1': 0.06165416950693024, 'rand_shuff_mean_RSA_1To1': -0.0014662957719399458, 'rand_shuff_pval_RSA_1To1': 0.178}
Nature: {'num_unq_pairs': 31, 'paired_SVCCA_1To1': 0.14806084711688297

...

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 355, 'paired_SVCCA_1To1': 0.12679035484936724, 'rand_shuff_mean_SVCCA_1To1': 0.038683547431560175, 'rand_shuff_pval_SVCCA_1To1': 0.01, 'paired_RSA_1To1': 0.01358948848902286, 'rand_shuff_mean_RSA_1To1': 0.0006527661172062771, 'rand_shuff_pval_RSA_1To1': 0.01}
Calendar: {'num_unq_pairs': 207, 'paired_SVCCA_1To1': 0.056808237676009794, 'rand_shuff_mean_SVCCA_1To1': 0.05949707521478104, 'rand_shuff_pval_SVCCA_1To1': 0.45, 'paired_RSA_1To1': 0.02503606164768246, 'rand_shuff_mean_RSA_1To1': 0.00028038244525870905, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 36, 'paired_SVCCA_1To1': 0.05149656611758357, 'rand_shuff_mean_SVCCA_1To1': 0.1341061450890076, 'rand_shuff_pval_SVCCA_1To1': 0.778, 'paired_RSA_1To1': 0.048036090265977466, 'rand_shuff_mean_RSA_1To1': 0.0007599211938169715, 'rand_shuff_pval_RSA_1To1': 0.136}
Nature: {'num_unq_pairs': 50, 'paired_SVCCA_1To1': 0.10463047730536262, 

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

sae.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Time
Calendar
People/Roles
Nature
Emotions
MonthNames
Countries
Time: {'num_unq_pairs': 371, 'paired_SVCCA_1To1': 0.20107250565767723, 'rand_shuff_mean_SVCCA_1To1': 0.04094027473741394, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.019901900534259224, 'rand_shuff_mean_RSA_1To1': 0.0006615199676735379, 'rand_shuff_pval_RSA_1To1': 0.0}
Calendar: {'num_unq_pairs': 189, 'paired_SVCCA_1To1': 0.2181168538418401, 'rand_shuff_mean_SVCCA_1To1': 0.05691357347961676, 'rand_shuff_pval_SVCCA_1To1': 0.0, 'paired_RSA_1To1': 0.02643991067956432, 'rand_shuff_mean_RSA_1To1': 0.0007948867919482135, 'rand_shuff_pval_RSA_1To1': 0.0}
People/Roles: {'num_unq_pairs': 29, 'paired_SVCCA_1To1': 0.41082769143417347, 'rand_shuff_mean_SVCCA_1To1': 0.15209594785986216, 'rand_shuff_pval_SVCCA_1To1': 0.03, 'paired_RSA_1To1': 0.09501530191185363, 'rand_shuff_mean_RSA_1To1': -0.0030138215241663495, 'rand_shuff_pval_RSA_1To1': 0.04}
Nature: {'num_unq_pairs': 39, 'paired_SVCCA_1To1': 0.3608755856084161, 'rand_sh

In [ ]:
layer_to_dictscores

{1: {'Time': {'num_unq_pairs': 277,
   'paired_SVCCA_1To1': 0.04721582249661352,
   'rand_shuff_mean_SVCCA_1To1': 0.05126210433138111,
   'rand_shuff_pval_SVCCA_1To1': 0.46,
   'paired_RSA_1To1': 0.013489543331351614,
   'rand_shuff_mean_RSA_1To1': 0.0007914899435218007,
   'rand_shuff_pval_RSA_1To1': 0.01},
  'Calendar': {'num_unq_pairs': 130,
   'paired_SVCCA_1To1': 0.10704092405185818,
   'rand_shuff_mean_SVCCA_1To1': 0.08042893493422304,
   'rand_shuff_pval_SVCCA_1To1': 0.31,
   'paired_RSA_1To1': 0.011259458409499375,
   'rand_shuff_mean_RSA_1To1': 0.0012132195117975817,
   'rand_shuff_pval_RSA_1To1': 0.18},
  'People/Roles': {'num_unq_pairs': 26,
   'paired_SVCCA_1To1': 0.014760603647810696,
   'rand_shuff_mean_SVCCA_1To1': 0.15932149387484248,
   'rand_shuff_pval_SVCCA_1To1': 0.954,
   'paired_RSA_1To1': 0.06165416950693024,
   'rand_shuff_mean_RSA_1To1': -0.0014662957719399458,
   'rand_shuff_pval_RSA_1To1': 0.178},
  'Nature': {'num_unq_pairs': 31,
   'paired_SVCCA_1To1': 0.14

In [ ]:
with open(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'concepts_layer_to_dictscores_L{layer_id_2}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L1

In [ ]:
all_scores_dict = layer_to_dictscores[1]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,130
Countries,20
Emotions,24
MonthNames,36
Nature,31
People/Roles,26
Time,277


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.05,0.05,0.46
Calendar,0.11,0.08,0.31
People/Roles,0.01,0.16,0.95
Nature,0.15,0.15,0.42
Emotions,0.31,0.16,0.13
MonthNames,0.03,0.14,0.88
Countries,0.41,0.20,0.09


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,0.0,0.01
Calendar,0.01,0.0,0.18
People/Roles,0.06,-0.0,0.18
Nature,0.10,0.0,0.05
Emotions,0.16,0.0,0.03
MonthNames,0.08,0.0,0.02
Countries,0.20,0.0,0.01


## analyze data: L2

In [ ]:
all_scores_dict = layer_to_dictscores[2]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,129
Countries,13
Emotions,22
MonthNames,26
Nature,29
People/Roles,25
Time,257


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.05,0.05,0.48
Calendar,0.09,0.07,0.35
People/Roles,0.62,0.16,0.00
Nature,0.17,0.15,0.38
Emotions,0.42,0.18,0.06
MonthNames,0.08,0.16,0.70
Countries,0.06,0.23,0.85


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,-0.0,0.02
Calendar,0.01,0.0,0.27
People/Roles,0.26,0.0,0.00
Nature,-0.01,0.0,0.60
Emotions,0.11,0.0,0.10
MonthNames,-0.01,-0.0,0.52
Countries,0.11,-0.0,0.17


## analyze data: L3

In [ ]:
all_scores_dict = layer_to_dictscores[3]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,161
Countries,18
Emotions,24
MonthNames,33
Nature,41
People/Roles,27
Time,290


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.13,0.05,0.02
Calendar,0.03,0.07,0.70
People/Roles,0.47,0.16,0.01
Nature,0.14,0.13,0.40
Emotions,0.32,0.17,0.12
MonthNames,0.21,0.14,0.23
Countries,0.12,0.20,0.64


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.03,0.0,0.00
Calendar,0.02,-0.0,0.04
People/Roles,0.12,0.0,0.06
Nature,0.02,0.0,0.30
Emotions,0.19,-0.0,0.00
MonthNames,-0.02,0.0,0.72
Countries,0.24,0.0,0.01


## analyze data: L4

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,207
Countries,11
Emotions,25
MonthNames,28
Nature,50
People/Roles,36
Time,355


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.13,0.04,0.01
Calendar,0.06,0.06,0.45
People/Roles,0.05,0.13,0.78
Nature,0.10,0.11,0.48
Emotions,0.30,0.16,0.15
MonthNames,0.11,0.15,0.56
Countries,0.34,0.26,0.31


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,0.0,0.01
Calendar,0.03,0.0,0.00
People/Roles,0.05,0.0,0.14
Nature,0.01,0.0,0.44
Emotions,0.08,0.0,0.09
MonthNames,-0.00,0.0,0.50
Countries,0.27,-0.0,0.02


## analyze data: L5

In [ ]:
all_scores_dict = layer_to_dictscores[4]

In [ ]:
filtered_words = ['num_unq_pairs']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,num_unq_pairs
Calendar,207
Countries,11
Emotions,25
MonthNames,28
Nature,50
People/Roles,36
Time,355


In [ ]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

""


In [ ]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
Time,0.13,0.04,0.01
Calendar,0.06,0.06,0.45
People/Roles,0.05,0.13,0.78
Nature,0.10,0.11,0.48
Emotions,0.30,0.16,0.15
MonthNames,0.11,0.15,0.56
Countries,0.34,0.26,0.31


In [ ]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
Time,0.01,0.0,0.01
Calendar,0.03,0.0,0.00
People/Roles,0.05,0.0,0.14
Nature,0.01,0.0,0.44
Emotions,0.08,0.0,0.09
MonthNames,-0.00,0.0,0.50
Countries,0.27,-0.0,0.02
